<a href="https://colab.research.google.com/github/duybluemind1988/Data-science/blob/master/Recommendation/Kaggle_movie_recommender_all_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install surprise

     |████████████████████████████████| 11.8MB 4.5MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1670921 sha256=e5e6e69c9774b6182b90d8d9d39a28da25a6e546ea2455f4f1b168b9f3311d53
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
#from surprise import Reader, Dataset, SVD
#from surprise.model_selection import cross_validate #replace for evaluate()

import warnings; warnings.simplefilter('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
movies_metadata=pd.read_csv('/content/drive/My Drive/Data/Movie dataset/movies_metadata.csv',encoding = "ISO-8859-1")
print(movies_metadata.shape)
movies_metadata.head(2)

(45466, 24)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [4]:
movies=pd.read_csv('/content/drive/My Drive/Data/Movie dataset/Small/movies.csv',encoding = "ISO-8859-1")
print(movies.shape)
movies.head()

(2500, 3)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings=pd.read_csv('/content/drive/My Drive/Data/Movie dataset/Small/ratings.csv',encoding = "ISO-8859-1")
print(ratings.shape)
ratings.head()

(264505, 4)


,userId,movieId,rating,timestamp
0,12882,1,4.0,1147195252
1,12882,32,3.5,1147195307
2,12882,47,5.0,1147195343
3,12882,50,5.0,1147185499
4,12882,110,4.5,1147195239


In [6]:
tags=pd.read_csv('/content/drive/My Drive/Data/Movie dataset/Small/tags.csv',encoding = "ISO-8859-1")
print(tags.shape)
tags.head()

(94875, 4)


,movieId,userId,tag,timestamp
0,3916,12882,sports,1147195545
1,4085,12882,Eddie Murphy,1147195966
2,33660,12882,boxing,1147195514
3,1197,320,must show,1145964801
4,1396,320,must show,1145964810


# Simple recommender

## Method 1: suggestion only by score

In [ ]:
df=movies_metadata.copy()
df.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [ ]:
#Calculate the number of votes garnered by the 80th percentile movie
m = df['vote_count'].quantile(0.80)
m

50.0

In [ ]:
#Only consider movies longer than 45 minutes and shorter than 300 minutes
q_movies = df[(df['runtime'] >= 45) & (df['runtime'] <= 300)]

#Only consider movies that have garnered more than m votes
q_movies = q_movies[q_movies['vote_count'] >= m]

#Inspect the number of movies that made the cut
print(q_movies.shape)
q_movies.head(2)

(8963, 24)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [ ]:
# Calculate C
C = df['vote_average'].mean()
C

5.618207215133889

In [ ]:
# Function to compute the IMDB weighted rating for each movie
# m: number of vote average (50)
# C: vote average (5.618) 
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Compute the weighted score
    return (v/(v+m) * R) + (m/(m+v) * C)

In [ ]:
# Compute the score using the weighted_rating function defined above
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

#Sort movies in descending order of their scores
q_movies = q_movies.sort_values('score', ascending=False)

#Print the top 25 movies
q_movies[['title', 'vote_count', 'vote_average', 'score', 'runtime']].head(25)

,title,vote_count,vote_average,score,runtime
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.855148,190.0
314,The Shawshank Redemption,8358.0,8.5,8.482863,142.0
834,The Godfather,6024.0,8.5,8.476278,175.0
40251,Your Name.,1030.0,8.5,8.366584,106.0
12481,The Dark Knight,12269.0,8.3,8.289115,152.0
2843,Fight Club,9678.0,8.3,8.286216,139.0
292,Pulp Fiction,8670.0,8.3,8.284623,154.0
522,Schindler's List,4436.0,8.3,8.270109,195.0
23673,Whiplash,4376.0,8.3,8.269704,105.0
5481,Spirited Away,3968.0,8.3,8.266628,125.0


## Method 2 (best) suggestion by genres and score

In [ ]:
md=movies_metadata.copy()
md.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [ ]:
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [ ]:
print(md.shape)

(45466, 24)


I use the TMDB Ratings to come up with our **Top Movies Chart.** I will use IMDB's *weighted rating* formula to construct my chart. Mathematically, it is represented as follows:

Weighted Rating (WR) = $(\frac{v}{v + m} . R) + (\frac{m}{v + m} . C)$

where,
* *v* is the number of votes for the movie
* *m* is the minimum votes required to be listed in the chart
* *R* is the average rating of the movie
* *C* is the mean vote across the whole report

The next step is to determine an appropriate value for *m*, the minimum votes required to be listed in the chart. We will use **95th percentile** as our cutoff. In other words, for a movie to feature in the charts, it must have more votes than at least 95% of the movies in the list.

I will build our overall Top 250 Chart and will define a function to build charts for a particular genre. Let's begin!

In [ ]:
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

5.244896612406511

In [ ]:
m = vote_counts.quantile(0.95)
m

434.0

In [ ]:
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').\
apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)
md['year'] 

0        1995
1        1995
2        1995
3        1995
4        1995
         ... 
45461     NaT
45462    2011
45463    2003
45464    1917
45465    2017
Name: year, Length: 45466, dtype: object

In [ ]:
qualified = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
print(qualified.shape)
qualified.head()

(2274, 6)


,title,year,vote_count,vote_average,popularity,genres
0,Toy Story,1995,5415,7,21.9469,"[Animation, Comedy, Family]"
1,Jumanji,1995,2413,6,17.0155,"[Adventure, Fantasy, Family]"
5,Heat,1995,1886,7,17.9249,"[Action, Crime, Drama, Thriller]"
9,GoldenEye,1995,1194,6,14.686,"[Adventure, Action, Thriller]"
15,Casino,1995,1343,7,10.1374,"[Drama, Crime]"


Filter top movie by weight score

In [ ]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [ ]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)
qualified = qualified.sort_values('wr', ascending=False).head(250)

In [ ]:
qualified.head(15)

,title,year,vote_count,vote_average,popularity,genres,wr
15480,Inception,2010,14075,8,29.1081,"[Action, Thriller, Science Fiction, Mystery, A...",7.917588
12481,The Dark Knight,2008,12269,8,123.167,"[Drama, Action, Crime, Thriller]",7.905871
22879,Interstellar,2014,11187,8,32.2135,"[Adventure, Drama, Science Fiction]",7.897107
2843,Fight Club,1999,9678,8,63.8696,[Drama],7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.0707,"[Adventure, Fantasy, Action]",7.871787
292,Pulp Fiction,1994,8670,8,140.95,"[Thriller, Crime]",7.868660
314,The Shawshank Redemption,1994,8358,8,51.6454,"[Drama, Crime]",7.864000
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.3244,"[Adventure, Fantasy, Action]",7.861927
351,Forrest Gump,1994,8147,8,48.3072,"[Comedy, Drama, Romance]",7.860656
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.4235,"[Adventure, Fantasy, Action]",7.851924


Filter top movie by genres

In [ ]:
s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)

In [ ]:
print(md.shape)
print(gen_md.shape) # rat nhieu film se duoc nhan them nhieu dong do co nhieu the loai

(45466, 25)
(93548, 25)


In [ ]:
gen_md.head()

(93548, 25)


,adult,belongs_to_collection,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,genre
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Animation
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Comedy
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Family
1,False,NaN,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,Adventure
1,False,NaN,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,Fantasy


In [ ]:
def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

In [ ]:
gen_md.genre.value_counts()

Drama                                    20265
Comedy                                   13182
Thriller                                  7624
Romance                                   6735
Action                                    6596
Horror                                    4673
Crime                                     4307
Documentary                               3932
Adventure                                 3496
Science Fiction                           3049
Family                                    2770
Mystery                                   2467
Fantasy                                   2313
Animation                                 1935
Foreign                                   1622
Music                                     1598
History                                   1398
War                                       1323
Western                                   1042
TV Movie                                   767
Vision View Entertainment                    1
BROSTA TV    

In [ ]:
build_chart('Romance').head(10)

,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457,8.565285
351,Forrest Gump,1994,8147,8,48.3072,7.971357
876,Vertigo,1958,1162,8,18.2082,7.811667
40251,Your Name.,2016,1030,8,34.461252,7.789489
883,Some Like It Hot,1959,835,8,11.8451,7.745154
1132,Cinema Paradiso,1988,834,8,14.177,7.744878
19901,Paperman,2012,734,8,7.19863,7.713951
37863,Sing Street,2016,669,8,10.672862,7.689483
882,The Apartment,1960,498,8,11.9943,7.599317
38718,The Handmaiden,2016,453,8,16.727405,7.566166


In [ ]:
build_chart('Action').head(10)

,title,year,vote_count,vote_average,popularity,wr
15480,Inception,2010,14075,8,29.1081,7.955099
12481,The Dark Knight,2008,12269,8,123.167,7.948610
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.0707,7.929579
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.3244,7.924031
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.4235,7.918382
256,Star Wars,1977,6778,8,42.1497,7.908327
1154,The Empire Strikes Back,1980,5998,8,19.471,7.896841
4135,Scarface,1983,3017,8,11.2997,7.802046
9430,Oldboy,2003,2000,8,10.6169,7.711649
1910,Seven Samurai,1954,892,8,15.0178,7.426145


### Explain

In [ ]:
genre='Romance'
percentile=0.85

df = gen_md[gen_md['genre'] == genre]
print(gen_md.shape)
print(df.shape)
df.head(2)

(93548, 25)
(6735, 25)


,adult,belongs_to_collection,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,genre
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,Romance
3,False,NaN,16000000,NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995,Romance


In [ ]:
# convert vote_count to int
vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
vote_counts

2         92
3         34
6        141
10       199
16       364
        ... 
45423      8
45425      4
45428      7
45437    146
45460     26
Name: vote_count, Length: 6735, dtype: int64

In [ ]:
# convert vote_averages to int
vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
vote_averages

2        6
3        6
6        6
10       6
16       7
        ..
45423    5
45425    6
45428    5
45437    8
45460    5
Name: vote_average, Length: 6735, dtype: int64

In [ ]:
C = vote_averages.mean()
m = vote_counts.quantile(percentile)
print(C) # trung bình review rating la 5.4
print(m) # khoảng 85% số lượng vote từ 90 trở lên, tối thiểu số này là OK

5.4041573867854495
90.89999999999964


In [ ]:
qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
print(df.shape) # romance movie only
print(qualified.shape) # roman movie with vote > 90 , average and vote not null

(6735, 25)
(1011, 5)


In [ ]:
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
qualified.head(2)

,title,year,vote_count,vote_average,popularity,wr
2,Grumpier Old Men,1995,92,6,11.7129,5.703870
6,Sabrina,1995,141,6,6.67728,5.766442


In [ ]:
qualified = qualified.sort_values('wr', ascending=False).head(250)
qualified.head()

,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457,8.565285
351,Forrest Gump,1994,8147,8,48.3072,7.971357
876,Vertigo,1958,1162,8,18.2082,7.811667
40251,Your Name.,2016,1030,8,34.461252,7.789489
883,Some Like It Hot,1959,835,8,11.8451,7.745154


### End

## Method 3: suggestion by genre, duration, year

In [ ]:
df=movies_metadata.copy()
#Only keep those features that we require 
df = df[['title','genres', 'release_date', 'runtime', 'vote_average', 'vote_count']]

df.head(2)

,title,genres,release_date,runtime,vote_average,vote_count
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",1995-10-30,81.0,7.7,5415.0
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",1995-12-15,104.0,6.9,2413.0


In [ ]:
#Convert release_date into pandas datetime format
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

#Extract year from the datetime
df['year'] = df['release_date'].apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)
df.head()

,title,genres,release_date,runtime,vote_average,vote_count,year
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",1995-10-30,81.0,7.7,5415.0,1995
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",1995-12-15,104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",1995-12-22,101.0,6.5,92.0,1995
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1995-12-22,127.0,6.1,34.0,1995
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",1995-02-10,106.0,5.7,173.0,1995


In [ ]:
#Helper function to convert NaT to 0 and all other years to integers.
def convert_int(x):
    try:
        return int(x)
    except:
        return 0

In [ ]:
#Apply convert_int to the year feature
df['year'] = df['year'].apply(convert_int)

In [ ]:
#Drop the release_date column
df = df.drop('release_date', axis=1)

In [ ]:
#Print genres of the first movie
df.iloc[0]['genres']

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [ ]:
#Import the literal_eval function from ast
from ast import literal_eval
#Convert all NaN into stringified empty lists
df['genres'] = df['genres'].fillna('[]')

#Apply literal_eval to convert stringified empty lists to the list object
df['genres'] = df['genres'].apply(literal_eval)

#Convert list of dictionaries to a list of strings
df['genres'] = df['genres'].apply(lambda x: [i['name'].lower() for i in x] if isinstance(x, list) else [])

In [ ]:
df

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"[animation, comedy, family]",81.0,7.7,5415.0,1995
1,Jumanji,"[adventure, fantasy, family]",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[romance, comedy]",101.0,6.5,92.0,1995
3,Waiting to Exhale,"[comedy, drama, romance]",127.0,6.1,34.0,1995
4,Father of the Bride Part II,[comedy],106.0,5.7,173.0,1995
...,...,...,...,...,...,...
45461,Subdue,"[drama, family]",90.0,4.0,1.0,0
45462,Century of Birthing,[drama],360.0,9.0,3.0,2011
45463,Betrayal,"[action, drama, thriller]",90.0,3.8,6.0,2003
45464,Satan Triumphant,[],87.0,0.0,0.0,1917


In [ ]:
#Create a new feature by exploding genres
s = df.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)

#Name the new feature as 'genre'
s.name = 'genre'

#Create a new dataframe gen_df which by dropping the old 'genres' feature and adding the new 'genre'.
gen_df = df.drop('genres', axis=1).join(s)

In [ ]:
#Print the head of the new gen_df
gen_df

,title,runtime,vote_average,vote_count,year,genre
0,Toy Story,81.0,7.7,5415.0,1995,animation
0,Toy Story,81.0,7.7,5415.0,1995,comedy
0,Toy Story,81.0,7.7,5415.0,1995,family
1,Jumanji,104.0,6.9,2413.0,1995,adventure
1,Jumanji,104.0,6.9,2413.0,1995,fantasy
...,...,...,...,...,...,...
45463,Betrayal,90.0,3.8,6.0,2003,action
45463,Betrayal,90.0,3.8,6.0,2003,drama
45463,Betrayal,90.0,3.8,6.0,2003,thriller
45464,Satan Triumphant,87.0,0.0,0.0,1917,NaN


In [ ]:
def build_chart(gen_df, percentile=0.8):
    #Ask for preferred genres
    print("Input preferred genre")
    genre = input()
    
    #Ask for lower limit of duration
    print("Input shortest duration")
    low_time = int(input())
    
    #Ask for upper limit of duration
    print("Input longest duration")
    high_time = int(input())
    
    #Ask for lower limit of timeline
    print("Input earliest year")
    low_year = int(input())
    
    #Ask for upper limit of timeline
    print("Input latest year")
    high_year = int(input())
    
    #Define a new movies variable to store the preferred movies. Copy the contents of gen_df to movies
    movies = gen_df.copy()
    
    #Filter based on the condition
    movies = movies[(movies['genre'] == genre) & 
                    (movies['runtime'] >= low_time) & 
                    (movies['runtime'] <= high_time) & 
                    (movies['year'] >= low_year) & 
                    (movies['year'] <= high_year)]
    
    #Compute the values of C and m for the filtered movies
    C = movies['vote_average'].mean()
    m = movies['vote_count'].quantile(percentile)
    
    #Only consider movies that have higher than m votes. Save this in a new dataframe q_movies
    q_movies = movies.copy().loc[movies['vote_count'] >= m]
    
    #Calculate score using the IMDB formula
    q_movies['score'] = q_movies.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) 
                                       + (m/(m+x['vote_count']) * C)
                                       ,axis=1)

    #Sort movies in descending order of their scores
    q_movies = q_movies.sort_values('score', ascending=False)
    
    return q_movies

In [ ]:
gen_df.describe()

,runtime,vote_average,vote_count,year
count,93194.000000,93527.000000,93527.000000,93548.000000
mean,96.753375,5.703193,146.629262,1989.223885
std,36.098436,1.743848,592.051356,77.263941
min,0.000000,0.000000,0.000000,0.000000
25%,87.000000,5.100000,4.000000,1979.000000
50%,96.000000,6.000000,13.000000,2001.000000
75%,108.000000,6.800000,48.000000,2010.000000
max,1256.000000,10.000000,14075.000000,2020.000000


In [ ]:
gen_df.genre.value_counts()

drama                                    20265
comedy                                   13182
thriller                                  7624
romance                                   6735
action                                    6596
horror                                    4673
crime                                     4307
documentary                               3932
adventure                                 3496
science fiction                           3049
family                                    2770
mystery                                   2467
fantasy                                   2313
animation                                 1935
foreign                                   1622
music                                     1598
history                                   1398
war                                       1323
western                                   1042
tv movie                                   767
the cartel                                   1
sentai filmwo

In [ ]:
#Generate the chart for top animation movies and display top 5.
build_chart(gen_df).head()

Input preferred genre
comedy
Input shortest duration
60
Input longest duration
120
Input earliest year
2005
Input latest year
2010


,title,runtime,vote_average,vote_count,year,genre,score
13724,Up,96.0,7.8,7048.0,2009,comedy,7.762541
15348,Toy Story 3,103.0,7.6,4710.0,2010,comedy,7.549669
11567,Ratatouille,111.0,7.5,4510.0,2007,comedy,7.450228
14310,Mary and Max,92.0,7.8,596.0,2009,comedy,7.428075
11073,Little Miss Sunshine,102.0,7.5,1728.0,2006,comedy,7.375567


In [ ]:
#Generate the chart for top animation movies and display top 5.
build_chart(gen_df).head()

Input preferred genre
adventure
Input shortest duration
60
Input longest duration
120
Input earliest year
2015
Input latest year
2020


,title,runtime,vote_average,vote_count,year,genre,score
36253,Zootopia,108.0,7.7,4961.0,2016,adventure,7.435592
26564,Deadpool,108.0,7.4,11444.0,2016,adventure,7.294943
38798,Captain Fantastic,119.0,7.9,1569.0,2016,adventure,7.187180
26553,Mad Max: Fury Road,120.0,7.3,9629.0,2015,adventure,7.184099
41457,Moana,107.0,7.3,3471.0,2016,adventure,7.016132


# Content Based Recommender

The recommender we built in the previous section suffers some severe limitations. For one, it gives the same recommendation to everyone, regardless of the user's personal taste. If a person who loves romantic movies (and hates action) were to look at our Top 15 Chart, s/he wouldn't probably like most of the movies. If s/he were to go one step further and look at our charts by genre, s/he wouldn't still be getting the best recommendations.

For instance, consider a person who loves *Dilwale Dulhania Le Jayenge*, *My Name is Khan* and *Kabhi Khushi Kabhi Gham*. One inference we can obtain is that the person loves the actor Shahrukh Khan and the director Karan Johar. Even if s/he were to access the romance chart, s/he wouldn't find these as the top recommendations.

To personalise our recommendations more, I am going to build an engine that computes similarity between movies based on certain metrics and suggests movies that are most similar to a particular movie that a user liked. Since we will be using movie metadata (or content) to build this engine, this also known as **Content Based Filtering.**

I will build two Content Based Recommenders based on:
* Movie Overviews and Taglines
* Movie Cast, Crew, Keywords and Genre

Also, as mentioned in the introduction, I will be using a subset of all the movies available to us due to limiting computing power available to me. 

In [ ]:
links_small = pd.read_csv('/content/drive/My Drive/Data/Movie dataset/links_small.csv')
links_small

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9120,162672,3859980,402672.0
9121,163056,4262980,315011.0
9122,163949,2531318,391698.0
9123,164977,27660,137608.0


In [ ]:
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')
links_small

0          862
1         8844
2        15602
3        31357
4        11862
         ...  
9120    402672
9121    315011
9122    391698
9123    137608
9124    410803
Name: tmdbId, Length: 9112, dtype: int64

In [ ]:
md.id

0           862
1          8844
2         15602
3         31357
4         11862
          ...  
45461    439050
45462    111109
45463     67758
45464    227506
45465    461257
Name: id, Length: 45466, dtype: object

In [ ]:
not_convert_list=[]
for i in range(len(md)):
  try:
    md['id'][i]=int(md['id'][i])
  except:
    print('cannot convert: ',md['id'][i])
    not_convert_list.append(md.index[i])

cannot convert:  1997-08-20
cannot convert:  2012-09-29
cannot convert:  2014-01-01


In [ ]:
not_convert_list

[19730, 29503, 35587]

In [ ]:
md = md.drop([19730, 29503, 35587])

In [ ]:
#Check EDA Notebook for how and why I got these indices.
md['id'] = md['id'].astype('int')

In [ ]:
print(md.shape)
smd = md[md['id'].isin(links_small)]
smd.shape

(45463, 25)


(9099, 25)

We have **9099** movies avaiable in our small movies metadata dataset which is 5 times smaller than our original dataset of 45000 movies.

## Movie Description Based Recommender

Let us first try to build a recommender using movie descriptions and taglines. We do not have a quantitative metric to judge our machine's performance so this will have to be done qualitatively.

In [ ]:
smd.head(5)

,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,description
0,0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,,Toy Story,False,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ..."
1,1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...
2,2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,A family wedding reignites the ancient feud be...
3,3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995,"Cheated on, mistreated and stepped on, the wom..."
4,4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995,Just when George Banks has recovered from his ...


In [ ]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

In [ ]:
smd['description'] # tag line + overview

0        Led by Woody, Andy's toys live happily in his ...
1        When siblings Judy and Peter discover an encha...
2        A family wedding reignites the ancient feud be...
3        Cheated on, mistreated and stepped on, the wom...
4        Just when George Banks has recovered from his ...
                               ...                        
40224    From the mind behind Evangelion comes a hit la...
40503    The band stormed Europe in 1963, and, in 1964,...
44821    When Molly Hale's sadness of her father's disa...
44826    All your favorite PokÃ©mon characters are back...
45265    While holidaying in the French Alps, a Swedish...
Name: description, Length: 9099, dtype: object

In [ ]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [ ]:
tfidf_matrix.shape

(9099, 268961)

#### Cosine Similarity

I will be using the Cosine Similarity to calculate a numeric quantity that denotes the similarity between two movies. Mathematically, it is defined as follows:

$cosine(x,y) = \frac{x. y^\intercal}{||x||.||y||} $

Since we have used the TF-IDF Vectorizer, calculating the Dot Product will directly give us the Cosine Similarity Score. Therefore, we will use sklearn's **linear_kernel** instead of cosine_similarities since it is much faster.

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
cosine_sim.shape

(9099, 9099)

In [ ]:
cosine_sim

array([[1.        , 0.00680459, 0.        , ..., 0.        , 0.00289435,
        0.        ],
       [0.00680459, 1.        , 0.01531013, ..., 0.00354933, 0.00640546,
        0.        ],
       [0.        , 0.01531013, 1.        , ..., 0.        , 0.00240445,
        0.00473472],
       ...,
       [0.        , 0.00354933, 0.        , ..., 1.        , 0.14629167,
        0.        ],
       [0.00289435, 0.00640546, 0.00240445, ..., 0.14629167, 1.        ,
        0.        ],
       [0.        , 0.        , 0.00473472, ..., 0.        , 0.        ,
        1.        ]])

We now have a pairwise cosine similarity matrix for all the movies in our dataset. The next step is to write a function that returns the 30 most similar movies based on the cosine similarity score.

In [ ]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])
indices

title
Toy Story                                                0
Jumanji                                                  1
Grumpier Old Men                                         2
Waiting to Exhale                                        3
Father of the Bride Part II                              4
                                                      ... 
Shin Godzilla                                         9094
The Beatles: Eight Days a Week - The Touring Years    9095
PokÃ©mon: Spell of the Unknown                        9096
PokÃ©mon 4Ever: Celebi - Voice of the Forest          9097
Force Majeure                                         9098
Length: 9099, dtype: int64

In [ ]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

We're all set. Let us now try and get the top recommendations for a few movies and see how good the recommendations are.

In [ ]:
get_recommendations('The Godfather').head(10)

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

In [ ]:
get_recommendations('The Dark Knight').head(10)

7931                      The Dark Knight Rises
132                              Batman Forever
1113                             Batman Returns
8227    Batman: The Dark Knight Returns, Part 2
7565                 Batman: Under the Red Hood
524                                      Batman
7901                           Batman: Year One
2579               Batman: Mask of the Phantasm
8165    Batman: The Dark Knight Returns, Part 1
2696                                        JFK
Name: title, dtype: object

Explain

In [ ]:
indices

title
Toy Story                                                0
Jumanji                                                  1
Grumpier Old Men                                         2
Waiting to Exhale                                        3
Father of the Bride Part II                              4
                                                      ... 
Shin Godzilla                                         9094
The Beatles: Eight Days a Week - The Touring Years    9095
Pokémon: Spell of the Unknown                         9096
Pokémon 4Ever: Celebi - Voice of the Forest           9097
Force Majeure                                         9098
Length: 9099, dtype: int64

In [ ]:
cosine_sim

array([[1.        , 0.00680476, 0.        , ..., 0.        , 0.00344913,
        0.        ],
       [0.00680476, 1.        , 0.01531062, ..., 0.00357057, 0.00762326,
        0.        ],
       [0.        , 0.01531062, 1.        , ..., 0.        , 0.00286535,
        0.00472155],
       ...,
       [0.        , 0.00357057, 0.        , ..., 1.        , 0.07811616,
        0.        ],
       [0.00344913, 0.00762326, 0.00286535, ..., 0.07811616, 1.        ,
        0.        ],
       [0.        , 0.        , 0.00472155, ..., 0.        , 0.        ,
        1.        ]])

In [ ]:
title='The Godfather'
idx = indices[title]
print('idx: ',idx)
sim_scores = list(enumerate(cosine_sim[idx])) # return (column/film id and value) tại row idx
print('sim_scores: ',sim_scores)
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) # sort value
print('sim_scores sorted: ',sim_scores)
sim_scores = sim_scores[1:31]
movie_indices = [i[0] for i in sim_scores]
print('movie_indices: ',movie_indices)
print(titles.iloc[movie_indices])# in tat ca ten phim co list id như trên

idx:  692
sim_scores:  [(0, 0.0), (1, 0.00282712634654008), (2, 0.013274406850346377), (3, 0.0), (4, 0.001911923808690747), (5, 0.009831202255696751), (6, 0.0), (7, 0.0048937541840076175), (8, 0.0), (9, 0.007200486377613863), (10, 0.0), (11, 0.0), (12, 0.002629662427098924), (13, 0.01655348503181121), (14, 0.0), (15, 0.003311321356213731), (16, 0.0), (17, 0.0017852909522175192), (18, 0.0), (19, 0.0), (20, 0.00138960229604634), (21, 0.02347433335488712), (22, 0.002909885967091068), (23, 0.0), (24, 0.0), (25, 0.0), (26, 0.0), (27, 0.011643868586706513), (28, 0.0), (29, 0.056141832299658065), (30, 0.0), (31, 0.0028878395660696552), (32, 0.0), (33, 0.0), (34, 0.0), (35, 0.0038924168745627535), (36, 0.0), (37, 0.0), (38, 0.02717387932966191), (39, 0.0), (40, 0.0), (41, 0.0020748439693151896), (42, 0.0), (43, 0.0), (44, 0.002612852388514977), (45, 0.0), (46, 0.01116205834731158), (47, 0.0), (48, 0.017220777855400132), (49, 0.003365386412223826), (50, 0.006507663461137723), (51, 0.00663189538

End

We see that for **The Dark Knight**, our system is able to identify it as a Batman film and subsequently recommend other Batman films as its top recommendations. But unfortunately, that is all this system can do at the moment. This is not of much use to most people as it doesn't take into considerations very important features such as cast, crew, director and genre, which determine the rating and the popularity of a movie. Someone who liked **The Dark Knight** probably likes it more because of Nolan and would hate **Batman Forever** and every other substandard movie in the Batman Franchise.

Therefore, we are going to use much more suggestive metadata than **Overview** and **Tagline**. In the next subsection, we will build a more sophisticated recommender that takes **genre**, **keywords**, **cast** and **crew** into consideration.

## Metadata Based Recommender

To build our standard metadata based content recommender, we will need to merge our current dataset with the crew and the keyword datasets. Let us prepare this data as our first step.

In [ ]:
credits = pd.read_csv('/content/drive/My Drive/Data/Movie dataset/credits.csv')
keywords = pd.read_csv('/content/drive/My Drive/Data/Movie dataset/keywords.csv')

In [ ]:
credits

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


In [ ]:
print(credits.cast[0]) # cac dien vien trong phim (id, nhan vat, gioi tinh, ten)
print(credits.crew[0]) # cac thanh vien trong doan lam phim (dao dien, nha san xuat...)

[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4tN

In [ ]:
keywords

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...
46414,439050,"[{'id': 10703, 'name': 'tragic love'}]"
46415,111109,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46416,67758,[]
46417,227506,[]


In [ ]:
credits[credits.id==862] # Toy story film

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862


In [ ]:
keywords.keywords[0] # Toy story film

"[{'id': 931, 'name': 'jealousy'}, {'id': 4290, 'name': 'toy'}, {'id': 5202, 'name': 'boy'}, {'id': 6054, 'name': 'friendship'}, {'id': 9713, 'name': 'friends'}, {'id': 9823, 'name': 'rivalry'}, {'id': 165503, 'name': 'boy next door'}, {'id': 170722, 'name': 'new toy'}, {'id': 187065, 'name': 'toy comes to life'}]"

In [ ]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

In [ ]:
md.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995


In [ ]:
md.shape

(45463, 25)

In [ ]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

In [ ]:
md.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."


In [ ]:
md.shape

(46628, 28)

In [ ]:
smd = md[md['id'].isin(links_small)]
smd.shape # loc 45000 phim thanh 9219 phim thôi

(9219, 28)

We now have our cast, crew, genres and credits, all in one dataframe. Let us wrangle this a little more using the following intuitions:

1. **Crew:** From the crew, we will only pick the director as our feature since the others don't contribute that much to the *feel* of the movie.
2. **Cast:** Choosing Cast is a little more tricky. Lesser known actors and minor roles do not really affect people's opinion of a movie. Therefore, we must only select the major characters and their respective actors. Arbitrarily we will choose the top 3 actors that appear in the credits list. 

In [ ]:
smd.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."


from ast import literal_eval

In [ ]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [ ]:
smd.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,cast,crew,keywords,cast_size,crew_size
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",13,106
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1...",26,16


In [ ]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [ ]:
smd['director'] = smd['crew'].apply(get_director)

In [ ]:
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [ ]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [ ]:
smd.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,cast,crew,keywords,cast_size,crew_size,director
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,"[Tom Hanks, Tim Allen, Don Rickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousy, toy, boy, friendship, friends, riva...",13,106,John Lasseter
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[board game, disappearance, based on children'...",26,16,Joe Johnston


My approach to building the recommender is going to be extremely *hacky*. What I plan on doing is creating a metadata dump for every movie which consists of **genres, director, main actors and keywords.** I then use a **Count Vectorizer** to create our count matrix as we did in the Description Recommender. The remaining steps are similar to what we did earlier: we calculate the cosine similarities and return movies that are most similar.

These are steps I follow in the preparation of my genres and credits data:
1. **Strip Spaces and Convert to Lowercase** from all our features. This way, our engine will not confuse between **Johnny Depp** and **Johnny Galecki.** 
2. **Mention Director 3 times** to give it more weight relative to the entire cast.

In [ ]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [ ]:
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x,x, x])

In [ ]:
smd.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,cast,crew,keywords,cast_size,crew_size,director
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,"[tomhanks, timallen, donrickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousy, toy, boy, friendship, friends, riva...",13,106,"[johnlasseter, johnlasseter, johnlasseter]"
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,"[robinwilliams, jonathanhyde, kirstendunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[board game, disappearance, based on children'...",26,16,"[joejohnston, joejohnston, joejohnston]"


#### Keywords

We will do a small amount of pre-processing of our keywords before putting them to any use. As a first step, we calculate the frequenct counts of every keyword that appears in the dataset.

In [ ]:
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [ ]:
s

0           jealousy
0                toy
0                boy
0         friendship
0            friends
            ...     
41391    destruction
41391          kaiju
41391          toyko
41669          music
41669    documentary
Name: keyword, Length: 64407, dtype: object

In [ ]:
s = s.value_counts()
s[:5]

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: keyword, dtype: int64

In [ ]:
s.shape

(12940,)

Keywords occur in frequencies ranging from 1 to 610. We do not have any use for keywords that occur only once. Therefore, these can be safely removed. Finally, we will convert every word to its stem so that words such as *Dogs* and *Dog* are considered the same.

In [ ]:
s = s[s > 1]
s

independent film         610
woman director           550
murder                   399
duringcreditsstinger     327
based on novel           318
                        ... 
strategic air command      2
translation                2
computer expert            2
live concert               2
sexual murder              2
Name: keyword, Length: 6709, dtype: int64

In [ ]:
#from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [ ]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [ ]:
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [ ]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

In [ ]:
smd.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,cast,crew,keywords,cast_size,crew_size,director,soup
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,"[tomhanks, timallen, donrickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousi, toy, boy, friendship, friend, rival...",13,106,"[johnlasseter, johnlasseter, johnlasseter]",jealousi toy boy friendship friend rivalri boy...
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,"[robinwilliams, jonathanhyde, kirstendunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[boardgam, disappear, basedonchildren'sbook, n...",26,16,"[joejohnston, joejohnston, joejohnston]",boardgam disappear basedonchildren'sbook newho...


In [ ]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

In [ ]:
count_matrix.shape

(9219, 107377)

In [ ]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [ ]:
cosine_sim

array([[1.        , 0.02441931, 0.02738955, ..., 0.        , 0.        ,
        0.        ],
       [0.02441931, 1.        , 0.        , ..., 0.02973505, 0.02500782,
        0.        ],
       [0.02738955, 0.        , 1.        , ..., 0.03335187, 0.        ,
        0.        ],
       ...,
       [0.        , 0.02973505, 0.03335187, ..., 1.        , 0.08700222,
        0.        ],
       [0.        , 0.02500782, 0.        , ..., 0.08700222, 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [ ]:
cosine_sim.shape

(9219, 9219)

In [ ]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [ ]:
indices

title
Toy Story                                                0
Jumanji                                                  1
Grumpier Old Men                                         2
Waiting to Exhale                                        3
Father of the Bride Part II                              4
                                                      ... 
The Last Brickmaker in America                        9214
Rustom                                                9215
Mohenjo Daro                                          9216
Shin Godzilla                                         9217
The Beatles: Eight Days a Week - The Touring Years    9218
Length: 9219, dtype: int64

In [ ]:
titles

0                                               Toy Story
1                                                 Jumanji
2                                        Grumpier Old Men
3                                       Waiting to Exhale
4                             Father of the Bride Part II
                              ...                        
9214                       The Last Brickmaker in America
9215                                               Rustom
9216                                         Mohenjo Daro
9217                                        Shin Godzilla
9218    The Beatles: Eight Days a Week - The Touring Y...
Name: title, Length: 9219, dtype: object

We will reuse the get_recommendations function that we had written earlier. Since our cosine similarity scores have changed, we expect it to give us different (and probably better) results. Let us check for **The Dark Knight** again and see what recommendations I get this time around.

In [ ]:
get_recommendations('The Dark Knight').head(10)

8031         The Dark Knight Rises
6218                 Batman Begins
6623                  The Prestige
2085                     Following
7648                     Inception
4145                      Insomnia
3381                       Memento
8613                  Interstellar
7659    Batman: Under the Red Hood
1134                Batman Returns
Name: title, dtype: object

I am much more satisfied with the results I get this time around. The recommendations seem to have recognized other Christopher Nolan movies (due to the high weightage given to director) and put them as top recommendations. I enjoyed watching **The Dark Knight** as well as some of the other ones in the list including **Batman Begins**, **The Prestige** and **The Dark Knight Rises**. 

We can of course experiment on this engine by trying out different weights for our features (directors, actors, genres), limiting the number of keywords that can be used in the soup, weighing genres based on their frequency, only showing movies with the same languages, etc.

Let me also get recommendations for another movie, **Mean Girls** which happens to be my girlfriend's favorite movie.

In [ ]:
get_recommendations('Mean Girls').head(10)

3319               Head Over Heels
4763                 Freaky Friday
1329              The House of Yes
6277              Just Like Heaven
7905         Mr. Popper's Penguins
7332    Ghosts of Girlfriends Past
6959     The Spiderwick Chronicles
8883                      The DUFF
6698         It's a Boy Girl Thing
7377       I Love You, Beth Cooper
Name: title, dtype: object

#### Popularity and Ratings

One thing that we notice about our recommendation system is that it recommends movies regardless of ratings and popularity. It is true that **Batman and Robin** has a lot of similar characters as compared to **The Dark Knight** but it was a terrible movie that shouldn't be recommended to anyone.

Therefore, we will add a mechanism to remove bad movies and return movies which are popular and have had a good critical response.

I will take the top 25 movies based on similarity scores and calculate the vote of the 60th percentile movie. Then, using this as the value of $m$, we will calculate the weighted rating of each movie using IMDB's formula like we did in the Simple Recommender section.

In [ ]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [ ]:
improved_recommendations('The Dark Knight')

,title,vote_count,vote_average,year,wr
7648,Inception,14075,8,2010,7.983343
8613,Interstellar,11187,8,2014,7.979077
6623,The Prestige,4510,8,2006,7.948714
3381,Memento,4168,8,2000,7.944596
8031,The Dark Knight Rises,9263,7,2012,6.984492
6218,Batman Begins,7511,7,2005,6.980918
1134,Batman Returns,1706,6,1992,5.969858
132,Batman Forever,1529,5,1995,5.022679
9024,Batman v Superman: Dawn of Justice,7189,5,2016,5.005046
1260,Batman & Robin,1447,4,1997,4.082995


### Explain:

In [ ]:
title='The Dark Knight'
idx = indices[title]
sim_scores = list(enumerate(cosine_sim[idx]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
sim_scores = sim_scores[1:26]
movie_indices = [i[0] for i in sim_scores]
movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
movies

,title,vote_count,vote_average,year
8031,The Dark Knight Rises,9263.0,7.6,2012
6218,Batman Begins,7511.0,7.5,2005
6623,The Prestige,4510.0,8.0,2006
2085,Following,363.0,7.2,1998
7648,Inception,14075.0,8.1,2010
4145,Insomnia,1181.0,6.8,2002
3381,Memento,4168.0,8.1,2000
8613,Interstellar,11187.0,8.1,2014
7659,Batman: Under the Red Hood,459.0,7.6,2010
1134,Batman Returns,1706.0,6.6,1992


In [ ]:

vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
m = vote_counts.quantile(0.60)
qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified['wr'] = qualified.apply(weighted_rating, axis=1)
qualified = qualified.sort_values('wr', ascending=False).head(10)
qualified

,title,vote_count,vote_average,year,wr
7648,Inception,14075,8,2010,7.872621
8613,Interstellar,11187,8,2014,7.843131
6623,The Prestige,4510,8,2006,7.662461
3381,Memento,4168,8,2000,7.641301
8031,The Dark Knight Rises,9263,7,2012,6.936548
6218,Batman Begins,7511,7,2005,6.923913
1134,Batman Returns,1706,6,1992,6.206438
132,Batman Forever,1529,5,1995,5.676520
9024,Batman v Superman: Dawn of Justice,7189,5,2016,5.224783
1260,Batman & Robin,1447,4,1997,5.167624


### End

Let me also get the recommendations for **Mean Girls**, my girlfriend's favorite movie.

In [ ]:
improved_recommendations('Mean Girls')

,title,vote_count,vote_average,year,wr
1547,The Breakfast Club,2189,7,1985,6.709602
390,Dazed and Confused,588,7,1993,6.254682
8883,The DUFF,1372,6,2015,5.818541
3712,The Princess Diaries,1063,6,2001,5.781086
4763,Freaky Friday,919,6,2003,5.757786
6277,Just Like Heaven,595,6,2005,5.681521
6959,The Spiderwick Chronicles,593,6,2008,5.680901
7494,American Pie Presents: The Book of Love,454,5,2009,5.119690
7332,Ghosts of Girlfriends Past,716,5,2009,5.092422
7905,Mr. Popper's Penguins,775,5,2011,5.087912


Unfortunately, **Batman and Robin** does not disappear from our recommendation list. This is probably due to the fact that it is rated a 4, which is only slightly below average on TMDB. It certainly doesn't deserve a 4 when amazing movies like **The Dark Knight Rises** has only a 7. However, there is nothing much we can do about this. Therefore, we will conclude our Content Based Recommender section here and come back to it when we build a hybrid engine.

# Collaborative Filtering

In [ ]:
movies=pd.read_csv('/content/drive/My Drive/Data/Movie dataset/Small/movies.csv',encoding = "ISO-8859-1")
print(movies.shape)
movies.head()

(2500, 3)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
titles=movies[['movieId','title']]
titles.set_index('movieId')

,title
movieId,
1,Toy Story (1995)
2,Jumanji (1995)
3,Grumpier Old Men (1995)
4,Waiting to Exhale (1995)
5,Father of the Bride Part II (1995)
...,...
109487,Interstellar (2014)
111362,X-Men: Days of Future Past (2014)
111759,Edge of Tomorrow (2014)


In [ ]:
ratings=pd.read_csv('/content/drive/My Drive/Data/Movie dataset/Small/ratings.csv',encoding = "ISO-8859-1")
print(ratings.shape)
ratings.head()

(264505, 4)


,userId,movieId,rating,timestamp
0,12882,1,4.0,1147195252
1,12882,32,3.5,1147195307
2,12882,47,5.0,1147195343
3,12882,50,5.0,1147185499
4,12882,110,4.5,1147195239


In [ ]:
print(len(ratings.userId.unique()))
print(len(ratings.movieId.unique()))

862
2500


## Normalize rating

In [ ]:
Mean = ratings.groupby(by="userId",as_index=False)['rating'].mean()
Mean.rename(columns={'rating':'avg_rating'},inplace=True)
Mean

,userId,avg_rating
0,316,3.329457
1,320,3.701613
2,359,3.685474
3,370,3.794404
4,910,3.898080
...,...,...
857,137609,3.711538
858,137805,2.779210
859,138072,2.432927
860,138176,4.107143


In [ ]:
rating_avg = pd.merge(ratings,Mean,on='userId')
rating_avg # rating_x: rating in each movie; rating_y: average rating all movie

,userId,movieId,rating,timestamp,avg_rating
0,12882,1,4.0,1147195252,4.061321
1,12882,32,3.5,1147195307,4.061321
2,12882,47,5.0,1147195343,4.061321
3,12882,50,5.0,1147185499,4.061321
4,12882,110,4.5,1147195239,4.061321
...,...,...,...,...,...
264500,90097,54286,5.0,1336143258,3.963415
264501,90097,60069,4.5,1336144698,3.963415
264502,90097,68358,5.0,1336144031,3.963415
264503,90097,68954,5.0,1336146596,3.963415


In [ ]:
rating_avg.columns

Index(['userId', 'movieId', 'rating', 'timestamp', 'avg_rating'], dtype='object')

In [ ]:
rating_avg['adg_rating']=rating_avg['rating']-rating_avg['avg_rating']
rating_avg.head() # phim nao co rating cao hon muc trung binh cua 1 user

,userId,movieId,rating,timestamp,avg_rating,adg_rating
0,12882,1,4.0,1147195252,4.061321,-0.061321
1,12882,32,3.5,1147195307,4.061321,-0.561321
2,12882,47,5.0,1147195343,4.061321,0.938679
3,12882,50,5.0,1147185499,4.061321,0.938679
4,12882,110,4.5,1147195239,4.061321,0.438679


## User-based Collaborative Filtering

In [ ]:
user_movie_matrix = rating_avg.pivot_table(
    index='userId', 
    columns='movieId', 
    values='adg_rating',
    aggfunc='mean'
)
user_movie_matrix

movieId,1,2,3,4,5,6,7,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,31,32,34,35,36,39,41,42,43,44,45,46,...,88140,88163,88744,88810,89492,89745,89864,90866,91500,91529,91542,91630,91658,92259,93840,94864,94959,95510,96079,96610,97304,97752,97913,97921,97938,98809,99114,102125,102445,104841,106487,106489,106782,106920,109374,109487,111362,111759,112556,112852
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
316,-0.829457,NaN,NaN,NaN,NaN,NaN,-1.329457,NaN,-0.829457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.329457,-1.329457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
359,1.314526,NaN,NaN,NaN,NaN,1.314526,NaN,NaN,0.314526,0.314526,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.685474,NaN,NaN,0.314526,NaN,-0.185474,NaN,NaN,NaN,NaN,-1.685474,NaN,-0.685474,0.314526,NaN,-0.685474,NaN,NaN,0.314526,NaN,-2.185474,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
370,0.705596,0.205596,NaN,NaN,NaN,1.205596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.705596,NaN,NaN,NaN,NaN,NaN,1.205596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.794404,0.205596,NaN,NaN,0.205596,-1.294404,NaN,NaN,-0.294404,NaN,NaN,NaN,NaN,-1.294404,0.705596,-2.794404,NaN,NaN,0.205596,0.205596,NaN,NaN,-0.294404,0.205596,NaN,0.205596,0.705596,-2.294404,NaN,NaN,-1.294404,-0.794404,0.705596,0.205596,NaN,NaN,-0.794404,0.705596,-0.294404,-0.794404
910,1.101920,0.101920,-0.398080,NaN,-0.39808,-0.398080,NaN,NaN,NaN,0.101920,NaN,NaN,NaN,0.10192,0.10192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.898080,NaN,NaN,NaN,NaN,NaN,0.101920,-0.898080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.398080,-1.898080,NaN,0.60192,0.601920,NaN,NaN,0.10192,NaN,NaN,-0.39808,NaN,0.10192,NaN,NaN,NaN,NaN,-0.398080,-0.398080,-1.898080,-0.89808,NaN,NaN,0.101920,NaN,-0.898080,NaN,NaN,-0.39808,NaN,NaN,NaN,-0.398080,NaN,NaN,NaN,NaN,0.601920,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137609,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.288462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
137805,1.220790,-0.779210,NaN,NaN,NaN,NaN,NaN,NaN,-1.279210,0.220790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.220790,NaN,NaN,1.220790,NaN,NaN,NaN,0.720790,NaN,0.72079,...,-0.779210,NaN,-1.77921,1.22079,NaN,2.220790,NaN,NaN,1.720790,1.72079,0.72079,1.22079,NaN,NaN,1.220790,NaN,NaN,-0.779210,1.220790,1.220790,NaN,NaN,1.220790,NaN,NaN,NaN,NaN,1.220790,-0.77921,1.72079,1.220790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.220790
138072,0.567073,-1.432927,-1.432927,NaN,NaN,NaN,NaN,-0.932927,0.567073,0.567073,-0.932927,NaN,NaN,NaN,NaN,NaN,0.067073,-1.432927,NaN,NaN,NaN,NaN,-0.432927,NaN,NaN,NaN,NaN,NaN,-0.432927,NaN,0.567073,NaN,NaN,0.567073,NaN,NaN,NaN,-0.432927,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [ ]:
# Replacing NaN by user Average (fill na by row)
user_movie_matrix = user_movie_matrix.apply(lambda row: row.fillna(row.mean()), axis=1)

In [ ]:
user_movie_matrix.head()

movieId,1,2,3,4,5,6,7,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,31,32,34,35,36,39,41,42,43,44,45,46,...,88140,88163,88744,88810,89492,89745,89864,90866,91500,91529,91542,91630,91658,92259,93840,94864,94959,95510,96079,96610,97304,97752,97913,97921,97938,98809,99114,102125,102445,104841,106487,106489,106782,106920,109374,109487,111362,111759,112556,112852
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
316,-8.294574e-01,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,-1.329457e+00,1.893404e-16,-8.294574e-01,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,-3.294574e-01,-1.329457e+00,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,...,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16
320,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,...,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17
359,1.314526e+00,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,1.314526e+00,-1.135546e-16,-1.135546e-16,3.145258e-01,3.145258e-01,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-6.854742e-01,-1.135546e-16,-1.135546e-16,3.145258e-01,-1.135546e-16,-1.854742e-01,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.685474e+00,-1.135546e-16,-6.854742e-01,3.145258e-01,-1.135546e-16,-6.854742e-01,-1.135546e-16,-1.135546e-16,3.145258e-01,-1.135546e-16,-2.185474e+00,-1.135546e-16,-1.135546e-16,...,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16
370,7.055961e-01,2.055961e-01,1.958963e-15,1.958963e-15,1.958963e-15,1.205596e+00,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,7.055961e-

In [ ]:
b=cosine_similarity(user_movie_matrix)
b

array([[ 1.00000000e+00,  6.00630917e-02,  7.20754753e-02, ...,
         7.73765811e-02,  2.69240918e-02, -2.27270553e-02],
       [ 6.00630917e-02,  1.00000000e+00,  6.30541209e-02, ...,
         9.88451426e-02,  3.87524588e-02,  5.66393007e-02],
       [ 7.20754753e-02,  6.30541209e-02,  1.00000000e+00, ...,
         1.02698453e-01,  9.93065563e-02,  3.14714649e-03],
       ...,
       [ 7.73765811e-02,  9.88451426e-02,  1.02698453e-01, ...,
         1.00000000e+00,  9.49195502e-02,  6.47542404e-02],
       [ 2.69240918e-02,  3.87524588e-02,  9.93065563e-02, ...,
         9.49195502e-02,  1.00000000e+00,  8.99631810e-04],
       [-2.27270553e-02,  5.66393007e-02,  3.14714649e-03, ...,
         6.47542404e-02,  8.99631810e-04,  1.00000000e+00]])

In [ ]:
np.fill_diagonal(b, 0 )
# Chuyen toan bo hang cheo 1 thanh 0 (user trung nhau khong co y nghia o day)

In [ ]:
user_user_sim_matrix = pd.DataFrame(b)
user_user_sim_matrix.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861
0,0.000000,0.060063,0.072075,0.043266,0.039305,0.045616,0.035341,0.038068,-1.248514e-02,0.050183,0.054092,0.033465,0.022670,0.126154,0.040799,0.037185,0.057644,0.025122,0.099331,0.074787,-0.014068,0.085619,0.025612,-0.077753,0.031668,0.023530,0.035985,0.028395,0.050490,0.061638,0.042351,0.050607,0.085542,0.064005,0.150968,0.089748,0.016363,0.152072,-0.024010,0.084487,...,0.098051,0.150762,0.044621,0.059883,0.025779,-0.012247,0.073318,0.115571,-0.003164,-0.029347,-0.028104,-0.014781,0.079841,0.086719,0.058335,0.030147,0.074533,0.042013,0.057231,0.062579,0.054507,0.019707,0.101334,0.019195,0.083581,0.081471,0.022737,0.096231,0.086519,-2.107738e-30,0.052632,0.104864,0.011358,0.029674,0.092552,0.017876,0.051371,0.077377,0.026924,-0.022727
1,0.060063,0.000000,0.063054,0.027315,0.006811,0.075620,0.011910,0.042509,8.859278e-32,0.067389,0.094096,0.027075,0.081181,0.112769,0.115683,-0.019041,0.067281,0.023103,0.178612,0.069101,0.030826,0.047998,0.054826,0.048060,-0.017636,0.035096,0.130301,0.098850,0.093140,0.175358,0.112067,0.045418,0.040291,0.113322,0.114264,0.078094,0.015466,0.143908,0.103990,0.097358,...,0.139018,0.099389,0.076827,0.086716,0.084780,0.041092,-0.020653,0.059496,0.039312,0.006270,0.052847,0.026312,0.057807,0.119811,0.036268,0.015588,0.080673,0.074018,0.159119,0.065811,0.075163,0.197053,0.122757,-0.007312,0.119167,0.084930,0.123965,0.148860,0.046513,-3.127799e-03,0.115325,0.065130,0.071996,0.097554,0.064769,-0.006251,0.077256,0.098845,0.038752,0.056639
2,0.072075,0.063054,0.000000,0.135836,0.076131,0.036757,0.046418,0.066544,4.287659e-02,0.109726,0.103606,0.074078,0.083495,0.108544,0.105273,0.090336,0.171797,0.042173,0.097642,0.063185,0.033466,0.091255,0.022727,0.016863,0.039266,0.071202,0.119431,0.054832,0.094266,0.059529,0.067309,0.108330,0.011763,0.087724,0.105287,0.084674,0.039859,0.082505,-0.023744,0.155337,...,0.093810,0.019195,0.069904,0.082246,0.030767,0.009669,-0.020447,0.029569,-0.031501,0.024651,0.054934,0.086611,0.064936,0.157955,0.048168,0.068332,0.074654,0.121211,0.096491,0.045586,0.061558,0.048304,0.056335,0.075844,0.112839,0.059213,0.085524,0.031384,0.086432,-1.150907e-02,0.120191,0.020672,0.032166,0.039599,0.108502,0.026371,0.075492,0.102698,0.099307,0.003147
3,0.043266,0.027315,0.135836,0.000000,0.108404,0.071655,0.070893,-0.003139,5.223516e-02,0.090241,0.130281,0.045340,0.024994,0.013804,0.100884,0.056301,0.136781,0.026465,0.076204,0.133776,0.059315,0.001900,0.019183,0.045782,0.027932,0.041196,0.098064,0.050647,0.075710,0.046873,0.047289,0.053318,0.061057,0.089613,0.137955,0.127379,0.030898,0.051115,0.022289,0.088772,...,0.097390,0.026579,0.130174,0.061356,0.028953,-0.030325,0.015702,0.051406,0.037940,0.032215,-0.135878,0.051708,0.031644,0.058526,-0.007105,0.079231,0.057712,0.037232,0.129099,0.029618,0.075959,0.049326,0.106989,0.015734,0.073179,0.015463,0.115510,0.050461,0.056026,9.514939e-03,0.091218,0.049594,0.004344,0.040692,0.110434,0.019767,-0.001364,0.052187,0.050997,0.009950
4,0.039305,0.006811,0.076131,0.108404,0.000000,0.021814,0.027339,-0.032211,-6.301121e-03,-0.007491,0.013776,0.035724,0.019892,0.025552,-0.003860,0.021981,0.030525,0.050039,0.047871,0.036584,0.019700,-0.019518,-0.024508,-0.024842,-0.035802,0.085419,0.088349,0.059115,0.026837,0.016708,0.044664,-0.034211,-0.012314,0.042145,0.111471,0.072073,0.052843,0.024620,-0.009871,0.065827,...,0.037647,0.044465,0.017315,0.066150,0.024728,-0.003301,0.001534,-0.007960,0.031310,0.007608,-0.009189,0.101635,0.027544,0.038293,0.016178,0.064773,0.044356,0.053461,0.099083,0.020399,0.049585,0.002393,0.056334,0.038195,0.043167,0.059144,0.060947,-0.003493,0.012221,-4.572897e-03,0.039464,-0.017620,0.020058,-0.004581,0.040866,-0.001438,-0.026082,0.073272,-0.012058,0.007610


In [ ]:
user_user_sim_matrix.columns = user_movie_matrix.index
user_user_sim_matrix['userId'] = user_movie_matrix.index
user_user_sim_matrix = user_user_sim_matrix.set_index('userId')
user_user_sim_matrix.head()

userId,316,320,359,370,910,975,1015,1387,1447,1588,1738,1741,1835,1935,2024,2248,2702,2847,2945,2988,3029,3076,3569,3665,3915,4215,4347,4483,4629,4931,5063,5268,5450,5475,5611,6116,6937,6988,7541,7723,...,131630,131835,131910,132187,132229,132295,132470,132578,132595,132914,133047,133811,134156,134181,134196,134279,134316,134401,134521,134525,134557,134627,134964,135092,135223,135418,136015,136037,136455,136856,137118,137209,137227,137446,137559,137609,137805,138072,138176,138200
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
316,0.000000,0.060063,0.072075,0.043266,0.039305,0.045616,0.035341,0.038068,-1.248514e-02,0.050183,0.054092,0.033465,0.022670,0.126154,0.040799,0.037185,0.057644,0.025122,0.099331,0.074787,-0.014068,0.085619,0.025612,-0.077753,0.031668,0.023530,0.035985,0.028395,0.050490,0.061638,0.042351,0.050607,0.085542,0.064005,0.150968,0.089748,0.016363,0.152072,-0.024010,0.084487,...,0.098051,0.150762,0.044621,0.059883,0.025779,-0.012247,0.073318,0.115571,-0.003164,-0.029347,-0.028104,-0.014781,0.079841,0.086719,0.058335,0.030147,0.074533,0.042013,0.057231,0.062579,0.054507,0.019707,0.101334,0.019195,0.083581,0.081471,0.022737,0.096231,0.086519,-2.107738e-30,0.052632,0.104864,0.011358,0.029674,0.092552,0.017876,0.051371,0.077377,0.026924,-0.022727
320,0.060063,0.000000,0.063054,0.027315,0.006811,0.075620,0.011910,0.042509,8.859278e-32,0.067389,0.094096,0.027075,0.081181,0.112769,0.115683,-0.019041,0.067281,0.023103,0.178612,0.069101,0.030826,0.047998,0.054826,0.048060,-0.017636,0.035096,0.130301,0.098850,0.093140,0.175358,0.112067,0.045418,0.040291,0.113322,0.114264,0.078094,0.015466,0.143908,0.103990,0.097358,...,0.139018,0.099389,0.076827,0.086716,0.084780,0.041092,-0.020653,0.059496,0.039312,0.006270,0.052847,0.026312,0.057807,0.119811,0.036268,0.015588,0.080673,0.074018,0.159119,0.065811,0.075163,0.197053,0.122757,-0.007312,0.119167,0.084930,0.123965,0.148860,0.046513,-3.127799e-03,0.115325,0.065130,0.071996,0.097554,0.064769,-0.006251,0.077256,0.098845,0.038752,0.056639
359,0.072075,0.063054,0.000000,0.135836,0.076131,0.036757,0.046418,0.066544,4.287659e-02,0.109726,0.103606,0.074078,0.083495,0.108544,0.105273,0.090336,0.171797,0.042173,0.097642,0.063185,0.033466,0.091255,0.022727,0.016863,0.039266,0.071202,0.119431,0.054832,0.094266,0.059529,0.067309,0.108330,0.011763,0.087724,0.105287,0.084674,0.039859,0.082505,-0.023744,0.155337,...,0.093810,0.019195,0.069904,0.082246,0.030767,0.009669,-0.020447,0.029569,-0.031501,0.024651,0.054934,0.086611,0.064936,0.157955,0.048168,0.068332,0.074654,0.121211,0.096491,0.045586,0.061558,0.048304,0.056335,0.075844,0.112839,0.059213,0.085524,0.031384,0.086432,-1.150907e-02,0.120191,0.020672,0.032166,0.039599,0.108502,0.026371,0.075492,0.102698,0.099307,0.003147
370,0.043266,0.027315,0.135836,0.000000,0.108404,0.071655,0.070893,-0.003139,5.223516e-02,0.090241,0.130281,0.045340,0.024994,0.013804,0.100884,0.056301,0.136781,0.026465,0.076204,0.133776,0.059315,0.001900,0.019183,0.045782,0.027932,0.041196,0.098064,0.050647,0.075710,0.046873,0.047289,0.053318,0.061057,0.089613,0.137955,0.127379,0.030898,0.051115,0.022289,0.088772,...,0.097390,0.026579,0.130174,0.061356,0.028953,-0.030325,0.015702,0.051406,0.037940,0.032215,-0.135878,0.051708,0.031644,0.058526,-0.007105,0.079231,0.057712,0.037232,0.129099,0.029618,0.075959,0.049326,0.106989,0.015734,0.073179,0.015463,0.115510,0.050461,0.056026,9.514939e-03,0.091218,0.049594,0.004344,0.040692,0.110434,0.019767,-0.001364,0.052187,0.050997,0.009950
910,0.039305,0.006811,0.076131,0.108404,0.000000,0.021814,0.027339,-0.032211,-6.301121e-03,-0.007491,0.013776,0.035724,0.019892,0.025552,-0.003860,0.021981,0.030525,0.050039,0.047871,0.036584,0.019700,-0.019518,-0.024508,-0.024842,-0.035802,0.085419,0.088349,0.059115,0.026837,0.016708,0.044664,-0.034211,-0.012314,0.042145,0.111471,0.072073,0.052843,0.024620,-0.009871,0.065827,...,0.037647,0.044465,0.017315,0.066150,0.024728,-0.003301,0.001534,-0.007960,

### Top 30 neighbours for each user

In [ ]:
df=user_user_sim_matrix.copy()
print(df.shape)
df.head()

(862, 862)


userId,316,320,359,370,910,975,1015,1387,1447,1588,1738,1741,1835,1935,2024,2248,2702,2847,2945,2988,3029,3076,3569,3665,3915,4215,4347,4483,4629,4931,5063,5268,5450,5475,5611,6116,6937,6988,7541,7723,...,131630,131835,131910,132187,132229,132295,132470,132578,132595,132914,133047,133811,134156,134181,134196,134279,134316,134401,134521,134525,134557,134627,134964,135092,135223,135418,136015,136037,136455,136856,137118,137209,137227,137446,137559,137609,137805,138072,138176,138200
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
316,0.000000,0.060063,0.072075,0.043266,0.039305,0.045616,0.035341,0.038068,-1.248514e-02,0.050183,0.054092,0.033465,0.022670,0.126154,0.040799,0.037185,0.057644,0.025122,0.099331,0.074787,-0.014068,0.085619,0.025612,-0.077753,0.031668,0.023530,0.035985,0.028395,0.050490,0.061638,0.042351,0.050607,0.085542,0.064005,0.150968,0.089748,0.016363,0.152072,-0.024010,0.084487,...,0.098051,0.150762,0.044621,0.059883,0.025779,-0.012247,0.073318,0.115571,-0.003164,-0.029347,-0.028104,-0.014781,0.079841,0.086719,0.058335,0.030147,0.074533,0.042013,0.057231,0.062579,0.054507,0.019707,0.101334,0.019195,0.083581,0.081471,0.022737,0.096231,0.086519,-2.107738e-30,0.052632,0.104864,0.011358,0.029674,0.092552,0.017876,0.051371,0.077377,0.026924,-0.022727
320,0.060063,0.000000,0.063054,0.027315,0.006811,0.075620,0.011910,0.042509,8.859278e-32,0.067389,0.094096,0.027075,0.081181,0.112769,0.115683,-0.019041,0.067281,0.023103,0.178612,0.069101,0.030826,0.047998,0.054826,0.048060,-0.017636,0.035096,0.130301,0.098850,0.093140,0.175358,0.112067,0.045418,0.040291,0.113322,0.114264,0.078094,0.015466,0.143908,0.103990,0.097358,...,0.139018,0.099389,0.076827,0.086716,0.084780,0.041092,-0.020653,0.059496,0.039312,0.006270,0.052847,0.026312,0.057807,0.119811,0.036268,0.015588,0.080673,0.074018,0.159119,0.065811,0.075163,0.197053,0.122757,-0.007312,0.119167,0.084930,0.123965,0.148860,0.046513,-3.127799e-03,0.115325,0.065130,0.071996,0.097554,0.064769,-0.006251,0.077256,0.098845,0.038752,0.056639
359,0.072075,0.063054,0.000000,0.135836,0.076131,0.036757,0.046418,0.066544,4.287659e-02,0.109726,0.103606,0.074078,0.083495,0.108544,0.105273,0.090336,0.171797,0.042173,0.097642,0.063185,0.033466,0.091255,0.022727,0.016863,0.039266,0.071202,0.119431,0.054832,0.094266,0.059529,0.067309,0.108330,0.011763,0.087724,0.105287,0.084674,0.039859,0.082505,-0.023744,0.155337,...,0.093810,0.019195,0.069904,0.082246,0.030767,0.009669,-0.020447,0.029569,-0.031501,0.024651,0.054934,0.086611,0.064936,0.157955,0.048168,0.068332,0.074654,0.121211,0.096491,0.045586,0.061558,0.048304,0.056335,0.075844,0.112839,0.059213,0.085524,0.031384,0.086432,-1.150907e-02,0.120191,0.020672,0.032166,0.039599,0.108502,0.026371,0.075492,0.102698,0.099307,0.003147
370,0.043266,0.027315,0.135836,0.000000,0.108404,0.071655,0.070893,-0.003139,5.223516e-02,0.090241,0.130281,0.045340,0.024994,0.013804,0.100884,0.056301,0.136781,0.026465,0.076204,0.133776,0.059315,0.001900,0.019183,0.045782,0.027932,0.041196,0.098064,0.050647,0.075710,0.046873,0.047289,0.053318,0.061057,0.089613,0.137955,0.127379,0.030898,0.051115,0.022289,0.088772,...,0.097390,0.026579,0.130174,0.061356,0.028953,-0.030325,0.015702,0.051406,0.037940,0.032215,-0.135878,0.051708,0.031644,0.058526,-0.007105,0.079231,0.057712,0.037232,0.129099,0.029618,0.075959,0.049326,0.106989,0.015734,0.073179,0.015463,0.115510,0.050461,0.056026,9.514939e-03,0.091218,0.049594,0.004344,0.040692,0.110434,0.019767,-0.001364,0.052187,0.050997,0.009950
910,0.039305,0.006811,0.076131,0.108404,0.000000,0.021814,0.027339,-0.032211,-6.301121e-03,-0.007491,0.013776,0.035724,0.019892,0.025552,-0.003860,0.021981,0.030525,0.050039,0.047871,0.036584,0.019700,-0.019518,-0.024508,-0.024842,-0.035802,0.085419,0.088349,0.059115,0.026837,0.016708,0.044664,-0.034211,-0.012314,0.042145,0.111471,0.072073,0.052843,0.024620,-0.009871,0.065827,...,0.037647,0.044465,0.017315,0.066150,0.024728,-0.003301,0.001534,-0.007960,

In [ ]:
n=30
order = np.argsort(df.values, axis=1)
print(order.shape)
order
# Returns the indices that would sort an array (index column id only)

(862, 862)


array([[682, 506, 332, ...,  44, 729, 704],
       [694, 720, 332, ..., 178, 704,  64],
       [332, 659, 730, ..., 637, 595, 634],
       ...,
       [691, 793, 332, ..., 207,  52, 417],
       [323, 290, 601, ..., 317, 254, 341],
       [318, 528, 447, ..., 557,  84,  64]])

In [ ]:
df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False).iloc[:n].index, 
                                  index=['top{}'.format(i) for i in range(1, n+1)]),
              axis=1)
# sort value tung row theo thu tu cao xuong thap, return index of each value
print(df.shape)
sim_user_30_u=df
sim_user_30_u.head()
# return column name with highest value (decending)

(862, 30)


,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,top11,top12,top13,top14,top15,top16,top17,top18,top19,top20,top21,top22,top23,top24,top25,top26,top27,top28,top29,top30
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
316,113673,117918,9050,12882,38187,102668,98880,43829,13215,78501,6988,5611,131835,86783,98781,94883,61305,59269,117861,128236,88608,120782,74472,53834,88928,42245,58265,89527,49830,63902
320,12288,113673,28159,79846,134627,112948,120729,97163,2945,4931,44400,61305,82880,21860,100540,12569,88608,124849,69256,59269,39271,94883,127683,101137,54989,134521,80946,10055,64365,106512
359,102118,96482,102532,50898,2702,60016,23428,120782,57937,42096,38159,32780,65670,124078,11343,46645,79531,35246,134181,128224,117258,7723,120729,61305,40768,117918,86768,129498,131620,58346
370,46645,42245,40768,23428,123707,60016,45120,113645,97195,102118,58265,113540,102532,120782,17039,117007,101137,57937,27365,41244,5611,20530,2702,38159,359,43354,117144,96482,2988,108195
910,87042,131620,67352,40768,31321,48821,26222,63295,5611,370,79531,84752,10164,17022,60016,133811,12271,88394,105455,35522,134521,88738,46645,108195,70201,58265,18115,114601,23428,17039


### All movie each user

In [ ]:
rating_avg

,userId,movieId,rating,timestamp,avg_rating,adg_rating
0,12882,1,4.0,1147195252,4.061321,-0.061321
1,12882,32,3.5,1147195307,4.061321,-0.561321
2,12882,47,5.0,1147195343,4.061321,0.938679
3,12882,50,5.0,1147185499,4.061321,0.938679
4,12882,110,4.5,1147195239,4.061321,0.438679
...,...,...,...,...,...,...
264500,90097,54286,5.0,1336143258,3.963415,1.036585
264501,90097,60069,4.5,1336144698,3.963415,0.536585
264502,90097,68358,5.0,1336144031,3.963415,1.036585
264503,90097,68954,5.0,1336146596,3.963415,1.036585


In [ ]:
rating_avg = rating_avg.astype({"movieId": str})
Movie_user = rating_avg.groupby(by = 'userId')['movieId'].apply(lambda x:','.join(x))
Movie_user


userId
316       1,7,10,32,34,150,160,163,165,185,196,223,253,2...
320       47,50,260,296,508,527,541,648,750,923,1136,119...
359       1,6,10,11,19,22,24,29,32,34,36,42,44,47,50,62,...
370       1,2,6,25,32,47,50,70,111,153,223,293,296,306,3...
910       1,2,3,5,6,11,15,16,25,32,34,52,62,95,100,110,1...
                                ...                        
137609    28,47,252,256,261,318,327,355,562,1042,1246,14...
137805    1,2,10,11,34,39,44,46,47,62,66,70,110,112,141,...
138072    1,2,3,9,10,11,12,18,19,24,31,34,39,44,50,63,65...
138176    44,318,502,527,711,858,1092,1221,1371,2023,239...
138200    1,32,39,47,110,165,215,253,342,355,364,367,374...
Name: movieId, Length: 862, dtype: object

### Make recommendation

In [ ]:
userId=370

#### list movie seen by this user

In [ ]:

Movie_seen_by_user=list(ratings[ratings.userId==userId].movieId.values)
print(len(Movie_seen_by_user))
print(Movie_seen_by_user[:10])

411
[1, 2, 6, 25, 32, 47, 50, 70, 111, 153]


#### Tim tat ca user id gan voi user da chon

In [ ]:
all_user_similar = sim_user_30_u[sim_user_30_u.index==userId].values
all_user_similar # tim tat ca user id gan voi user da chon

array([[ 46645,  42245,  40768,  23428, 123707,  60016,  45120, 113645,
         97195, 102118,  58265, 113540, 102532, 120782,  17039, 117007,
        101137,  57937,  27365,  41244,   5611,  20530,   2702,  38159,
           359,  43354, 117144,  96482,   2988, 108195]])

In [ ]:
all_user_similar = all_user_similar.squeeze().tolist()
print(all_user_similar)

[46645, 42245, 40768, 23428, 123707, 60016, 45120, 113645, 97195, 102118, 58265, 113540, 102532, 120782, 17039, 117007, 101137, 57937, 27365, 41244, 5611, 20530, 2702, 38159, 359, 43354, 117144, 96482, 2988, 108195]


#### Movies_under_consideration

In [ ]:
d = Movie_user[Movie_user.index.isin(all_user_similar)]
d # tim tat ca cac phim cua cac user similar voi user da chon

userId
359       1,6,10,11,19,22,24,29,32,34,36,42,44,47,50,62,...
2702      1,6,12,19,32,44,47,50,110,111,150,153,158,165,...
2988      1,2,18,21,25,29,32,34,36,47,50,69,73,82,85,110...
5611      1,32,47,110,111,145,150,172,193,223,253,260,29...
17039     1,29,32,47,50,111,150,260,296,318,431,541,589,...
20530     1,6,11,21,25,32,47,50,105,110,141,153,185,191,...
23428     1,2,47,50,70,107,150,153,223,231,260,296,318,3...
27365     6,18,32,47,50,70,111,153,223,288,296,316,318,3...
38159     1,2,5,6,10,11,16,19,22,39,42,47,50,61,65,70,10...
40768     1,2,10,13,17,34,44,47,48,50,110,111,150,158,18...
41244     1,7,11,32,34,47,50,60,62,73,104,110,150,153,15...
42245     10,16,21,32,34,42,44,47,50,110,111,150,151,153...
43354     2,6,10,16,17,31,34,47,50,88,160,161,165,170,18...
45120     32,50,76,175,193,215,231,260,293,296,318,319,3...
46645     1,2,10,16,19,32,34,36,47,50,52,58,70,97,110,11...
57937     1,6,10,16,21,32,34,47,50,104,110,111,150,153,1...
58265     6,10,11,15,19,21,32,47,

In [ ]:
l = ','.join(d.values)
Movie_seen_by_similar_users = l.split(',')
print(len(Movie_seen_by_similar_users))
print(len(set(Movie_seen_by_similar_users)))
# List tat ca phim ma nhung similar users đã xem

15313
2268


In [ ]:
Movies_under_consideration = list(set(Movie_seen_by_similar_users)-set(list(map(str, Movie_seen_by_user))))
print(len(Movies_under_consideration))
print(Movies_under_consideration)
# Lấy list movie by similar user - movie seen by user được list cac moive tiềm năng

1857
['3638', '1208', '4867', '2375', '2416', '73', '6942', '350', '432', '3363', '2421', '4223', '2335', '3623', '2973', '3146', '3871', '3681', '4754', '2001', '2072', '4159', '2398', '40278', '940', '261', '207', '3968', '1210', '586', '5064', '7156', '6155', '2163', '1405', '2318', '546', '4039', '7', '902', '6863', '1120', '4047', '109487', '2382', '2360', '2094', '81', '1334', '2606', '3384', '71057', '4214', '2598', '1425', '233', '1556', '4865', '5970', '2707', '38038', '4010', '932', '80489', '5995', '2348', '5294', '2961', '5446', '8977', '47629', '2405', '3639', '2278', '468', '1307', '4386', '5254', '910', '1991', '64622', '585', '5283', '8810', '3896', '3298', '533', '1488', '17', '1848', '2087', '44199', '8370', '524', '1086', '5464', '1188', '2021', '8969', '780', '407', '2408', '7569', '520', '3258', '4247', '502', '2067', '3633', '429', '3916', '46', '3802', '46970', '1213', '8947', '33166', '3246', '82459', '1608', '1894', '4887', '1873', '3809', '1799', '3897', '2947

In [ ]:
Movies_under_consideration = list(map(int, Movies_under_consideration))
print(len(Movies_under_consideration))
# Chuyển list này thành int

1857


#### Tính score từng moive under này (Very hard way)

In [ ]:
Mean.head()

,userId,avg_rating
0,316,3.329457
1,320,3.701613
2,359,3.685474
3,370,3.794404
4,910,3.898080


In [ ]:
user_movie_matrix.head()

movieId,1,2,3,4,5,6,7,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,31,32,34,35,36,39,41,42,43,44,45,46,...,88140,88163,88744,88810,89492,89745,89864,90866,91500,91529,91542,91630,91658,92259,93840,94864,94959,95510,96079,96610,97304,97752,97913,97921,97938,98809,99114,102125,102445,104841,106487,106489,106782,106920,109374,109487,111362,111759,112556,112852
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
316,-8.294574e-01,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,-1.329457e+00,1.893404e-16,-8.294574e-01,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,-3.294574e-01,-1.329457e+00,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,...,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16
320,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,...,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17
359,1.314526e+00,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,1.314526e+00,-1.135546e-16,-1.135546e-16,3.145258e-01,3.145258e-01,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-6.854742e-01,-1.135546e-16,-1.135546e-16,3.145258e-01,-1.135546e-16,-1.854742e-01,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.685474e+00,-1.135546e-16,-6.854742e-01,3.145258e-01,-1.135546e-16,-6.854742e-01,-1.135546e-16,-1.135546e-16,3.145258e-01,-1.135546e-16,-2.185474e+00,-1.135546e-16,-1.135546e-16,...,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16
370,7.055961e-01,2.055961e-01,1.958963e-15,1.958963e-15,1.958963e-15,1.205596e+00,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,7.055961e-

In [ ]:
user_user_sim_matrix.head()

userId,316,320,359,370,910,975,1015,1387,1447,1588,1738,1741,1835,1935,2024,2248,2702,2847,2945,2988,3029,3076,3569,3665,3915,4215,4347,4483,4629,4931,5063,5268,5450,5475,5611,6116,6937,6988,7541,7723,...,131630,131835,131910,132187,132229,132295,132470,132578,132595,132914,133047,133811,134156,134181,134196,134279,134316,134401,134521,134525,134557,134627,134964,135092,135223,135418,136015,136037,136455,136856,137118,137209,137227,137446,137559,137609,137805,138072,138176,138200
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
316,0.000000,0.060063,0.072075,0.043266,0.039305,0.045616,0.035341,0.038068,-1.248514e-02,0.050183,0.054092,0.033465,0.022670,0.126154,0.040799,0.037185,0.057644,0.025122,0.099331,0.074787,-0.014068,0.085619,0.025612,-0.077753,0.031668,0.023530,0.035985,0.028395,0.050490,0.061638,0.042351,0.050607,0.085542,0.064005,0.150968,0.089748,0.016363,0.152072,-0.024010,0.084487,...,0.098051,0.150762,0.044621,0.059883,0.025779,-0.012247,0.073318,0.115571,-0.003164,-0.029347,-0.028104,-0.014781,0.079841,0.086719,0.058335,0.030147,0.074533,0.042013,0.057231,0.062579,0.054507,0.019707,0.101334,0.019195,0.083581,0.081471,0.022737,0.096231,0.086519,-2.107738e-30,0.052632,0.104864,0.011358,0.029674,0.092552,0.017876,0.051371,0.077377,0.026924,-0.022727
320,0.060063,0.000000,0.063054,0.027315,0.006811,0.075620,0.011910,0.042509,8.859278e-32,0.067389,0.094096,0.027075,0.081181,0.112769,0.115683,-0.019041,0.067281,0.023103,0.178612,0.069101,0.030826,0.047998,0.054826,0.048060,-0.017636,0.035096,0.130301,0.098850,0.093140,0.175358,0.112067,0.045418,0.040291,0.113322,0.114264,0.078094,0.015466,0.143908,0.103990,0.097358,...,0.139018,0.099389,0.076827,0.086716,0.084780,0.041092,-0.020653,0.059496,0.039312,0.006270,0.052847,0.026312,0.057807,0.119811,0.036268,0.015588,0.080673,0.074018,0.159119,0.065811,0.075163,0.197053,0.122757,-0.007312,0.119167,0.084930,0.123965,0.148860,0.046513,-3.127799e-03,0.115325,0.065130,0.071996,0.097554,0.064769,-0.006251,0.077256,0.098845,0.038752,0.056639
359,0.072075,0.063054,0.000000,0.135836,0.076131,0.036757,0.046418,0.066544,4.287659e-02,0.109726,0.103606,0.074078,0.083495,0.108544,0.105273,0.090336,0.171797,0.042173,0.097642,0.063185,0.033466,0.091255,0.022727,0.016863,0.039266,0.071202,0.119431,0.054832,0.094266,0.059529,0.067309,0.108330,0.011763,0.087724,0.105287,0.084674,0.039859,0.082505,-0.023744,0.155337,...,0.093810,0.019195,0.069904,0.082246,0.030767,0.009669,-0.020447,0.029569,-0.031501,0.024651,0.054934,0.086611,0.064936,0.157955,0.048168,0.068332,0.074654,0.121211,0.096491,0.045586,0.061558,0.048304,0.056335,0.075844,0.112839,0.059213,0.085524,0.031384,0.086432,-1.150907e-02,0.120191,0.020672,0.032166,0.039599,0.108502,0.026371,0.075492,0.102698,0.099307,0.003147
370,0.043266,0.027315,0.135836,0.000000,0.108404,0.071655,0.070893,-0.003139,5.223516e-02,0.090241,0.130281,0.045340,0.024994,0.013804,0.100884,0.056301,0.136781,0.026465,0.076204,0.133776,0.059315,0.001900,0.019183,0.045782,0.027932,0.041196,0.098064,0.050647,0.075710,0.046873,0.047289,0.053318,0.061057,0.089613,0.137955,0.127379,0.030898,0.051115,0.022289,0.088772,...,0.097390,0.026579,0.130174,0.061356,0.028953,-0.030325,0.015702,0.051406,0.037940,0.032215,-0.135878,0.051708,0.031644,0.058526,-0.007105,0.079231,0.057712,0.037232,0.129099,0.029618,0.075959,0.049326,0.106989,0.015734,0.073179,0.015463,0.115510,0.050461,0.056026,9.514939e-03,0.091218,0.049594,0.004344,0.040692,0.110434,0.019767,-0.001364,0.052187,0.050997,0.009950
910,0.039305,0.006811,0.076131,0.108404,0.000000,0.021814,0.027339,-0.032211,-6.301121e-03,-0.007491,0.013776,0.035724,0.019892,0.025552,-0.003860,0.021981,0.030525,0.050039,0.047871,0.036584,0.019700,-0.019518,-0.024508,-0.024842,-0.035802,0.085419,0.088349,0.059115,0.026837,0.016708,0.044664,-0.034211,-0.012314,0.042145,0.111471,0.072073,0.052843,0.024620,-0.009871,0.065827,...,0.037647,0.044465,0.017315,0.066150,0.024728,-0.003301,0.001534,-0.007960,

In [ ]:
score = []
for item in Movies_under_consideration:
  #print('-'*50)
  #print('item: ',item)
  c = user_movie_matrix.loc[:,item] 
  #print('c loc ra rating của movie đó theo toàn bộ user \n',c) # loc ra toan bo user theo colum movie dang chọn
  #print(len(c))
  d = c[c.index.isin(b)]  
  #print('d loc cac user id/rating average lien quan voi user da chon\n',d) # loc cac user id lien quan voi user da chon (b)
  #print(len(d))
  f = d[d.notnull()] 
  #print('f loai bo null user\n',f) # loai bo null user
  #print(len(f))
  avg_user = Mean.loc[Mean['userId'] == userId,'avg_rating'].values[0] 
  #print('avg_user\n',avg_user)#loc mean rating value cua user đang chon
  index = f.index.values.squeeze().tolist()
  #print('index return index cac user id similar to user đã chọn\n',index) # return index cac user id similar to user đã chọn
  #print(len(index))
  corr = user_user_sim_matrix.loc[userId,index]
  #print('corr: trong danh sach similarity user x user loc ra row=user, column= list user similarity\n',corr) # trong danh sach similarity user x user loc ra row=user, column= list user similarity
  fin = pd.concat([f, corr], axis=1)
  #print('fin\n',fin)6. 
  fin.columns = ['adg_score','correlation']
  #print('fin\n',fin)
  fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
  #print('fin\n',fin)
  nume = fin['score'].sum()
  #print('nume\n',nume)
  deno = fin['correlation'].sum()
  #print('deno\n',deno)
  final_score = avg_user + (nume/deno)
  #print('final_score\n',final_score)
  score.append(final_score)
#print('list_score',score)

In [ ]:
data = pd.DataFrame({'movieId':Movies_under_consideration,'score':score})
data

,movieId,score
0,3638,3.681502
1,1208,4.331430
2,4867,3.780960
3,2375,3.805372
4,2416,3.714898
...,...,...
1852,227,3.756604
1853,3104,3.811394
1854,63082,4.111716
1855,67255,3.791701


In [ ]:
top_5_recommendation = data.sort_values(by='score',ascending=False).head(5)
top_5_recommendation

,movieId,score
476,1196,4.629604
1508,260,4.465601
114,1213,4.454659
1373,1258,4.438358
863,1201,4.402002


In [ ]:
Movie_Name = top_5_recommendation.merge(movies, how='inner', on='movieId')
Movie_Name

,movieId,score,title,genres
0,1196,4.629604,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi
1,260,4.465601,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi
2,1213,4.454659,Goodfellas (1990),Crime|Drama
3,1258,4.438358,"Shining, The (1980)",Horror
4,1201,4.402002,"Good, the Bad and the Ugly, The (Buono, il bru...",Action|Adventure|Western


In [ ]:
Movie_Names = Movie_Name.title.values.tolist()
print(Movie_Names)

['Star Wars: Episode V - The Empire Strikes Back (1980)', 'Star Wars: Episode IV - A New Hope (1977)', 'Goodfellas (1990)', 'Shining, The (1980)', 'Good, the Bad and the Ugly, The (Buono, il brutto, il cattivo, Il) (1966)']


#### Explain

In [ ]:
score = []
for item in Movies_under_consideration[:1]:
  print('-'*50)
  print('item: ',item)
  c = user_movie_matrix.loc[:,item] 
  print('c loc ra rating của movie đó theo toàn bộ user \n',c) # loc ra toan bo user theo colum movie dang chọn
  print(len(c))
  d = c[c.index.isin(b)]  
  print('d loc cac user id/rating average lien quan voi user da chon\n',d) # loc cac user id lien quan voi user da chon (b)
  print(len(d))
  f = d[d.notnull()] 
  print('f loai bo null user\n',f) # loai bo null user
  print(len(f))
  avg_user = Mean.loc[Mean['userId'] == userId,'avg_rating'].values[0] 
  print('avg_user\n',avg_user)#loc mean rating value cua user đang chon
  index = f.index.values.squeeze().tolist()
  print('index return index cac user id similar to user đã chọn\n',index) # return index cac user id similar to user đã chọn
  print(len(index))
  corr = user_user_sim_matrix.loc[userId,index]
  print('corr: trong danh sach similarity user x user loc ra row=user, column= list user similarity\n',corr) # trong danh sach similarity user x user loc ra row=user, column= list user similarity
  fin = pd.concat([f, corr], axis=1)
  print('fin\n',fin)
  fin.columns = ['adg_score','correlation']
  print('fin\n',fin)
  fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
  print('fin\n',fin)
  nume = fin['score'].sum()
  print('nume\n',nume)
  deno = fin['correlation'].sum()
  print('deno\n',deno)
  final_score = avg_user + (nume/deno)
  print('final_score\n',final_score)
  score.append(final_score)
print('list_score',score)

--------------------------------------------------
item:  3638
c loc ra rating của movie đó theo toàn bộ user 
 userId
316       1.893404e-16
320       4.297638e-17
359      -1.135546e-16
370       1.958963e-15
910       6.795811e-16
              ...     
137609   -1.366428e-16
137805   -1.046891e-15
138072   -1.932927e+00
138176    3.806479e-16
138200    7.808746e-16
Name: 3638, Length: 862, dtype: float64
862
d loc cac user id/rating average lien quan voi user da chon
 userId
359      -1.135546e-16
2702      8.242099e-17
2988      2.616565e-17
5611      6.647373e-16
17039    -6.767074e-17
20530    -3.724619e-17
23428     9.754102e-16
27365    -1.384174e-16
38159     0.000000e+00
40768    -5.113137e-16
41244    -3.384390e-16
42245    -9.296593e-16
43354    -5.589623e-01
45120     0.000000e+00
46645    -2.012422e-01
57937    -1.164244e+00
58265     2.139073e-01
60016     4.791753e-16
96482    -9.421488e-01
97195     5.423712e-16
101137    5.717887e-18
102118    1.843972e-02
102532    

#### End

#### Tính score từng movie này để recommend (easy way)

Score của các movie này theo 30 user similarity

In [ ]:
rating_avg

,userId,movieId,rating,timestamp,avg_rating,adg_rating
0,12882,1,4.0,1147195252,4.061321,-0.061321
1,12882,32,3.5,1147195307,4.061321,-0.561321
2,12882,47,5.0,1147195343,4.061321,0.938679
3,12882,50,5.0,1147185499,4.061321,0.938679
4,12882,110,4.5,1147195239,4.061321,0.438679
...,...,...,...,...,...,...
264500,90097,54286,5.0,1336143258,3.963415,1.036585
264501,90097,60069,4.5,1336144698,3.963415,0.536585
264502,90097,68358,5.0,1336144031,3.963415,1.036585
264503,90097,68954,5.0,1336146596,3.963415,1.036585


In [ ]:
print(len(Movies_under_consideration))
print(Movies_under_consideration[:10])

1857
[3638, 1208, 4867, 2375, 2416, 73, 6942, 350, 432, 3363]


In [ ]:
print(all_user_similar)

[46645, 42245, 40768, 23428, 123707, 60016, 45120, 113645, 97195, 102118, 58265, 113540, 102532, 120782, 17039, 117007, 101137, 57937, 27365, 41244, 5611, 20530, 2702, 38159, 359, 43354, 117144, 96482, 2988, 108195]


In [ ]:
# Loc dach sach cac movie rating co user similar và under consideration
rating_avg_filter=rating_avg[rating_avg.userId.isin(all_user_similar) & 
                            rating_avg.movieId.isin(list(map(str, Movies_under_consideration)))]
rating_avg_filter
# adg_rating: gia tri rating cua user tren movie do - gia tri rating trung binh cua user do tren toan bo movie
# neu adg_rating >0 : phim nay hay hon muc trung binh cua nguoi do, neu < 0 thi phim nay do hon muc trung binh cua nguoi do
# adg_rating average của các movie sẽ cho ta biết mưc độ hay dở thế nào...

,userId,movieId,rating,timestamp,avg_rating,adg_rating
3877,108195,5,2.0,1095774393,2.924271,-0.924271
3879,108195,7,2.0,1095774409,2.924271,-0.924271
3880,108195,10,3.5,1095772767,2.924271,0.575729
3881,108195,11,3.0,1095755740,2.924271,0.075729
3882,108195,14,2.0,1095774403,2.924271,-0.924271
...,...,...,...,...,...,...
260569,20530,91658,3.0,1367429314,3.439785,-0.439785
260572,20530,94959,4.0,1367427353,3.439785,0.560215
260575,20530,97304,3.5,1367427378,3.439785,0.060215
260577,20530,97938,3.0,1367427365,3.439785,-0.439785


In [ ]:
movie_top_10=rating_avg_filter.groupby('movieId')['adg_rating'].mean().sort_values(ascending=False)[:10]
movie_top_10

movieId
2132    1.560215
1267    1.350755
123     1.314526
930     1.298758
3526    1.234043
1131    1.234043
521     1.234043
1132    1.234043
82      1.234043
898     1.149194
Name: adg_rating, dtype: float64

In [ ]:
list(movie_top_10.index)

['2132', '1267', '123', '930', '3526', '1131', '521', '1132', '82', '898']

In [ ]:
movies_names=movies.set_index('movieId')['title']
movies_names

movieId
1                           Toy Story (1995)
2                             Jumanji (1995)
3                    Grumpier Old Men (1995)
4                   Waiting to Exhale (1995)
5         Father of the Bride Part II (1995)
                         ...                
109487                   Interstellar (2014)
111362     X-Men: Days of Future Past (2014)
111759               Edge of Tomorrow (2014)
112556                      Gone Girl (2014)
112852        Guardians of the Galaxy (2014)
Name: title, Length: 2500, dtype: object

In [ ]:
list(map(int,list(movie_top_10.index)))

[2132, 1267, 123, 930, 3526, 1131, 521, 1132, 82, 898]

In [ ]:
movies_names.loc[list(map(int,list(movie_top_10.index)))]

movieId
2132            Who's Afraid of Virginia Woolf? (1966)
1267                  Manchurian Candidate, The (1962)
123      Chungking Express (Chung Hing sam lam) (1994)
930                                   Notorious (1946)
3526                                 Parenthood (1989)
1131                           Jean de Florette (1986)
521                           Romeo Is Bleeding (1993)
1132    Manon of the Spring (Manon des sources) (1986)
82                     Antonia's Line (Antonia) (1995)
898                     Philadelphia Story, The (1940)
Name: title, dtype: object

## Item-based Collaborative Filtering

In [24]:
%matplotlib inline
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import warnings; warnings.simplefilter('ignore')

In [25]:
movies=pd.read_csv('/content/drive/My Drive/Data/Movie dataset/Small/movies.csv',encoding = "ISO-8859-1")
print(movies.shape)
movies.head()

(2500, 3)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [44]:
titles=movies[['movieId','title']]
titles=titles.set_index('movieId')['title']
titles

movieId
1                           Toy Story (1995)
2                             Jumanji (1995)
3                    Grumpier Old Men (1995)
4                   Waiting to Exhale (1995)
5         Father of the Bride Part II (1995)
                         ...                
109487                   Interstellar (2014)
111362     X-Men: Days of Future Past (2014)
111759               Edge of Tomorrow (2014)
112556                      Gone Girl (2014)
112852        Guardians of the Galaxy (2014)
Name: title, Length: 2500, dtype: object

In [27]:
ratings=pd.read_csv('/content/drive/My Drive/Data/Movie dataset/Small/ratings.csv',encoding = "ISO-8859-1")
print(ratings.shape)
ratings.head()

(264505, 4)


,userId,movieId,rating,timestamp
0,12882,1,4.0,1147195252
1,12882,32,3.5,1147195307
2,12882,47,5.0,1147195343
3,12882,50,5.0,1147185499
4,12882,110,4.5,1147195239


In [28]:
print(len(ratings.userId.unique()))
print(len(ratings.movieId.unique()))

862
2500


In [29]:
Mean = ratings.groupby(by="userId",as_index=False)['rating'].mean()
Mean.rename(columns={'rating':'avg_rating'},inplace=True)
Mean

,userId,avg_rating
0,316,3.329457
1,320,3.701613
2,359,3.685474
3,370,3.794404
4,910,3.898080
...,...,...
857,137609,3.711538
858,137805,2.779210
859,138072,2.432927
860,138176,4.107143


In [30]:
rating_avg = pd.merge(ratings,Mean,on='userId')
rating_avg # rating_x: rating in each movie; rating_y: average rating all movie

,userId,movieId,rating,timestamp,avg_rating
0,12882,1,4.0,1147195252,4.061321
1,12882,32,3.5,1147195307,4.061321
2,12882,47,5.0,1147195343,4.061321
3,12882,50,5.0,1147185499,4.061321
4,12882,110,4.5,1147195239,4.061321
...,...,...,...,...,...
264500,90097,54286,5.0,1336143258,3.963415
264501,90097,60069,4.5,1336144698,3.963415
264502,90097,68358,5.0,1336144031,3.963415
264503,90097,68954,5.0,1336146596,3.963415


In [31]:
rating_avg['adg_rating']=rating_avg['rating']-rating_avg['avg_rating']
rating_avg.head() # phim nao co rating cao hon muc trung binh cua 1 user

,userId,movieId,rating,timestamp,avg_rating,adg_rating
0,12882,1,4.0,1147195252,4.061321,-0.061321
1,12882,32,3.5,1147195307,4.061321,-0.561321
2,12882,47,5.0,1147195343,4.061321,0.938679
3,12882,50,5.0,1147185499,4.061321,0.938679
4,12882,110,4.5,1147195239,4.061321,0.438679


In [32]:
user_movie_matrix = rating_avg.pivot_table(
    index='userId', 
    columns='movieId', 
    values='adg_rating',
    aggfunc='mean'
)
user_movie_matrix

movieId,1,2,3,4,5,6,7,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,31,32,34,35,36,39,41,42,43,44,45,46,...,88140,88163,88744,88810,89492,89745,89864,90866,91500,91529,91542,91630,91658,92259,93840,94864,94959,95510,96079,96610,97304,97752,97913,97921,97938,98809,99114,102125,102445,104841,106487,106489,106782,106920,109374,109487,111362,111759,112556,112852
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
316,-0.829457,NaN,NaN,NaN,NaN,NaN,-1.329457,NaN,-0.829457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.329457,-1.329457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
359,1.314526,NaN,NaN,NaN,NaN,1.314526,NaN,NaN,0.314526,0.314526,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.685474,NaN,NaN,0.314526,NaN,-0.185474,NaN,NaN,NaN,NaN,-1.685474,NaN,-0.685474,0.314526,NaN,-0.685474,NaN,NaN,0.314526,NaN,-2.185474,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
370,0.705596,0.205596,NaN,NaN,NaN,1.205596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.705596,NaN,NaN,NaN,NaN,NaN,1.205596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.794404,0.205596,NaN,NaN,0.205596,-1.294404,NaN,NaN,-0.294404,NaN,NaN,NaN,NaN,-1.294404,0.705596,-2.794404,NaN,NaN,0.205596,0.205596,NaN,NaN,-0.294404,0.205596,NaN,0.205596,0.705596,-2.294404,NaN,NaN,-1.294404,-0.794404,0.705596,0.205596,NaN,NaN,-0.794404,0.705596,-0.294404,-0.794404
910,1.101920,0.101920,-0.398080,NaN,-0.39808,-0.398080,NaN,NaN,NaN,0.101920,NaN,NaN,NaN,0.10192,0.10192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.898080,NaN,NaN,NaN,NaN,NaN,0.101920,-0.898080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.398080,-1.898080,NaN,0.60192,0.601920,NaN,NaN,0.10192,NaN,NaN,-0.39808,NaN,0.10192,NaN,NaN,NaN,NaN,-0.398080,-0.398080,-1.898080,-0.89808,NaN,NaN,0.101920,NaN,-0.898080,NaN,NaN,-0.39808,NaN,NaN,NaN,-0.398080,NaN,NaN,NaN,NaN,0.601920,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137609,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.288462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
137805,1.220790,-0.779210,NaN,NaN,NaN,NaN,NaN,NaN,-1.279210,0.220790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.220790,NaN,NaN,1.220790,NaN,NaN,NaN,0.720790,NaN,0.72079,...,-0.779210,NaN,-1.77921,1.22079,NaN,2.220790,NaN,NaN,1.720790,1.72079,0.72079,1.22079,NaN,NaN,1.220790,NaN,NaN,-0.779210,1.220790,1.220790,NaN,NaN,1.220790,NaN,NaN,NaN,NaN,1.220790,-0.77921,1.72079,1.220790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.220790
138072,0.567073,-1.432927,-1.432927,NaN,NaN,NaN,NaN,-0.932927,0.567073,0.567073,-0.932927,NaN,NaN,NaN,NaN,NaN,0.067073,-1.432927,NaN,NaN,NaN,NaN,-0.432927,NaN,NaN,NaN,NaN,NaN,-0.432927,NaN,0.567073,NaN,NaN,0.567073,NaN,NaN,NaN,-0.432927,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [33]:
# Replacing NaN by user Average (fill na by row)
user_movie_matrix = user_movie_matrix.apply(lambda row: row.fillna(row.mean()), axis=1)

In [34]:
user_movie_matrix.T

userId,316,320,359,370,910,975,1015,1387,1447,1588,1738,1741,1835,1935,2024,2248,2702,2847,2945,2988,3029,3076,3569,3665,3915,4215,4347,4483,4629,4931,5063,5268,5450,5475,5611,6116,6937,6988,7541,7723,...,131630,131835,131910,132187,132229,132295,132470,132578,132595,132914,133047,133811,134156,134181,134196,134279,134316,134401,134521,134525,134557,134627,134964,135092,135223,135418,136015,136037,136455,136856,137118,137209,137227,137446,137559,137609,137805,138072,138176,138200
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,-8.294574e-01,4.297638e-17,1.314526e+00,7.055961e-01,1.101920e+00,1.651571e-17,4.468085e-01,-6.443299e-01,1.113636e+00,1.003534e+00,2.272084e-16,9.641186e-01,8.575130e-01,0.511719,-2.327044e-01,-1.156697e-16,6.246537e-01,-4.878727e-16,-5.362814e-16,-7.659574e-01,5.340237e-01,-2.613240e-02,-0.375,9.162304e-01,-3.750087e-17,8.585434e-01,5.842207e-01,-1.246819e-01,-5.741627e-02,-8.114587e-17,3.910311e-16,-4.229421e-17,8.881784e-16,7.142857e-02,4.654088e-01,-1.570260e-16,-1.467391e-01,-1.694030e+00,1.268826e-16,6.686508e-01,...,-6.896024e-01,0.0,4.587097e-01,1.201500e+00,-2.354208e-16,1.575931e-01,4.099285e-16,3.275862e-01,6.096939e-01,-4.898190e-01,1.124276e-16,7.302251e-01,-7.034483e-01,1.707345e+00,-7.836868e-17,1.75,9.868649e-17,-6.755745e-16,-8.956421e-17,-0.5,2.917688e-16,5.228311e-01,4.072327e-01,1.350091e+00,4.417144e-16,-0.335938,1.294971e+00,6.832142e-17,-9.273399e-01,-5.231736e-16,-9.522145e-01,3.525594e-16,-1.937844e-16,7.656711e-18,-3.186275e-01,-1.366428e-16,1.220790e+00,5.670732e-01,3.806479e-16,-4.222488e-01
2,1.893404e-16,4.297638e-17,-1.135546e-16,2.055961e-01,1.019202e-01,1.651571e-17,-5.531915e-01,-6.443299e-01,1.009294e-16,2.510752e-17,2.272084e-16,9.641186e-01,-1.424870e-01,0.000000,-2.327044e-01,-1.156697e-16,8.242099e-17,-4.878727e-16,-5.362814e-16,-1.265957e+00,-1.064237e-16,-1.841346e-16,0.000,3.441110e-16,-3.750087e-17,-6.468526e-16,-8.099362e-16,6.327989e-17,-1.057416e+00,-8.114587e-17,-5.400982e-02,-4.229421e-17,8.881784e-16,1.824829e-16,6.647373e-16,-1.570260e-16,-3.427210e-16,1.060512e-16,1.268826e-16,-9.868649e-17,...,-1.189602e+00,0.0,-4.016859e-16,-3.530509e-16,-2.354208e-16,-6.362310e-16,4.099285e-16,-1.225074e-16,-3.903061e-01,-9.898190e-01,1.124276e-16,7.302251e-01,-2.034483e-01,-2.926554e-01,-7.836868e-17,0.00,9.868649e-17,-1.777211e-01,-8.956421e-17,0.0,2.917688e-16,-5.677853e-17,-1.550123e-16,2.825284e-16,-1.985294e+00,0.164062,-2.050290e-01,6.832142e-17,-4.273399e-01,-5.231736e-16,-8.281384e-18,2.671756e-01,-1.937844e-16,-3.060345e-01,-1.175530e-16,-1.366428e-16,-7.792096e-01,-1.432927e+00,3.806479e-16,7.808746e-16
3,1.893404e-16,4.297638e-17,-1.135546e-16,1.958963e-15,-3.980798e-01,1.651571e-17,8.188879e-17,3.456571e-16,1.009294e-16,2.510752e-17,2.272084e-16,2.133845e-16,7.041000e-16,0.000000,3.360927e-16,-1.156697e-16,8.242099e-17,-4.878727e-16,-5.362814e-16,2.616565e-17,-1.064237e-16,-1.841346e-16,-1.875,3.441110e-16,-3.750087e-17,-6.468526e-16,-8.099362e-16,6.327989e-17,-5.482058e-16,-8.114587e-17,4.459902e-01,-4.229421e-17,8.881784e-16,1.824829e-16,6.647373e-16,-1.570260e-16,-3.427210e-16,1.060512e-16,1.268826e-16,-9.868649e-17,...,1.887719e-16,0.0,-4.016859e-16,-3.530509e-16,-2.354208e-16,-6.362310e-16,4.099285e-16,-1.225074e-16,-7.250436e-17,5.465713e-16,1.124276e-16,-7.697749e-01,-2.034483e-01,-2.769912e-16,-7.836868e-17,0.00,9.868649e-17,-6.755745e-16,-8.956421e-17,0.0,2.917688e-16,-5.677853e-17,-1.550123e-16,2.825284e-16,4.417144e-16,-0.835938,8.864605e-16,6.832142e-17,-4.273399e-01,-5.231736e-16,-8.281384e-18,3.525594e-16,-1.937844e-16,7.656711e-18,-1.175530e-16,-1.366428e-16,-1.046891e-15,-1.432927e+00,3.806479e-16,7.808746e-16
4,1.893404e-16,4.297638e-17,-1.135546e-16,1.958963e-15,6.795811e-16,1.651571e-17,8.188879e-17,3.456571e-16,1.009294e-16,2.510752e-17,2.272084e-16,2.133845e-16,7.041000e-16,0.000000,3.360927e-16,-1.156697e-16,8.242099e-17,-4.878727e-16,-5.362814e-16,2.616565e-17,-1.064237e-16,-1.841346e-16,0

In [35]:
b=cosine_similarity(user_movie_matrix.T)
b

array([[ 1.        , -0.00727701, -0.03159433, ...,  0.11531286,
         0.08126606,  0.08911925],
       [-0.00727701,  1.        ,  0.14100306, ..., -0.0733531 ,
        -0.07703056, -0.10773674],
       [-0.03159433,  0.14100306,  1.        , ..., -0.03206176,
        -0.02001517, -0.05869697],
       ...,
       [ 0.11531286, -0.0733531 , -0.03206176, ...,  1.        ,
         0.14491174,  0.2992348 ],
       [ 0.08126606, -0.07703056, -0.02001517, ...,  0.14491174,
         1.        ,  0.25652769],
       [ 0.08911925, -0.10773674, -0.05869697, ...,  0.2992348 ,
         0.25652769,  1.        ]])

In [36]:
np.fill_diagonal(b, 0 )
# Chuyen toan bo hang cheo 1 thanh 0 (user trung nhau khong co y nghia o day)

In [37]:
item_item_sim_matrix = pd.DataFrame(b)
print(item_item_sim_matrix.shape)
item_item_sim_matrix.head()

(2500, 2500)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2460,2461,2462,2463,2464,2465,2466,2467,2468,2469,2470,2471,2472,2473,2474,2475,2476,2477,2478,2479,2480,2481,2482,2483,2484,2485,2486,2487,2488,2489,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0.000000,-0.007277,-0.031594,-0.112114,-0.084925,0.131804,-0.054632,-0.087031,0.130114,-0.010739,-0.127292,-0.096353,0.011103,-0.116477,0.046306,0.067544,-0.069773,-0.075580,-0.139184,0.099152,-0.029994,-0.097387,-0.107096,0.052382,-0.023883,-0.070794,0.037616,-0.013090,-0.143816,0.134021,0.104742,0.067002,0.062423,-0.047229,-0.007889,0.008920,-0.037446,-0.147649,-0.027022,-0.092771,...,-0.052596,0.032802,0.010607,-0.008083,0.086686,0.068732,0.082161,0.092138,-0.033797,0.079142,0.009608,0.049843,0.053826,0.017696,0.067804,-0.081896,0.040345,-0.044197,0.084057,0.040065,0.058572,0.020407,0.098499,0.047683,0.040197,-0.026742,0.128330,0.008891,-0.032677,0.038396,0.103224,0.004923,0.061771,-2.853563e-02,0.068199,-0.016748,0.043112,0.115313,0.081266,0.089119
1,-0.007277,0.000000,0.141003,0.017774,0.157326,-0.115083,0.095137,0.086244,0.088405,-0.057072,0.126449,-0.017812,0.038624,0.094194,-0.086791,-0.131311,-0.001828,0.211701,0.063678,0.047955,0.097420,0.035511,0.111818,-0.063916,-0.033053,-0.006995,-0.089168,-0.045563,0.092575,-0.192623,0.046540,0.031395,-0.039137,0.040389,0.018714,0.023227,-0.005611,0.114926,0.042039,-0.006644,...,-0.049050,-0.016471,0.007191,-0.006670,-0.118413,-0.133636,-0.096270,0.010410,-0.037918,-0.043505,-0.093290,-0.015636,-0.033560,-0.018887,-0.060303,0.039396,-0.099282,0.057432,-0.090076,-0.078073,-0.074336,-0.010221,-0.057673,0.017165,0.002624,-0.037733,-0.113207,0.007571,0.004953,-0.153065,-0.079466,-0.021880,-0.107930,-5.311594e-02,-0.082836,-0.135727,-0.064378,-0.073353,-0.077031,-0.107737
2,-0.031594,0.141003,0.000000,0.090117,0.168187,-0.058197,0.098877,0.096959,0.100318,-0.126105,0.119754,-0.122850,0.032978,0.037591,-0.019851,-0.050918,0.026399,0.199380,0.001943,0.070301,0.029817,0.076066,0.038338,-0.054217,0.002246,-0.046317,-0.089217,-0.029053,-0.005643,-0.056524,-0.001915,-0.011630,-0.024059,0.037106,0.007963,0.029700,-0.022484,0.168235,-0.034380,-0.066714,...,-0.012139,-0.005233,0.001949,-0.050156,-0.090194,-0.043842,-0.037210,-0.084131,-0.038575,-0.050541,-0.072345,-0.023248,-0.046942,-0.043564,-0.043742,-0.036170,-0.030675,-0.003238,-0.033010,-0.107466,-0.037243,0.002773,-0.019546,-0.013317,-0.013576,-0.054247,-0.092227,-0.033111,-0.060398,-0.040722,-0.006139,-0.021662,-0.034608,-5.433816e-03,-0.073513,-0.077483,-0.042966,-0.032062,-0.020015,-0.058697
3,-0.112114,0.017774,0.090117,0.000000,0.077437,-0.021579,0.175584,0.073419,0.009188,0.057196,-0.000597,0.010305,0.000238,0.161127,-0.048875,-0.082975,-0.025624,0.152856,0.028948,0.038174,-0.043365,0.012787,-0.048001,-0.186289,-0.017436,0.023842,-0.036042,-0.126064,-0.007546,-0.071052,-0.045153,-0.149667,-0.058492,-0.005848,-0.039359,-0.021668,0.081075,0.054547,0.026525,0.127764,...,0.008995,-0.023751,-0.018682,-0.009570,-0.017160,-0.019711,-0.009586,-0.026425,-0.026799,0.008910,-0.024172,0.000584,-0.029833,-0.008420,-0.013855,-0.036805,-0.124496,0.010260,-0.028603,-0.015729,-0.054039,0.018653,0.012379,-0.049026,-0.011943,-0.015531,-0.121384,0.009509,0.009081,-0.001081,0.006465,0.000134,0.000002,-2.503878e-16,-0.126685,-0.040177,-0.023114,0.006900,0.000470,-0.021005
4,-0.084925,0.157326,0.168187,0.077437,0.000000,-0.120131,0.091224,0.124185,0.034659,0.275990,0.129333,-0.036621,0.033024,0.057356,-0.151201,-0.115386,-0.016743,0.172960,0.052055,0.038355,0.156915,0.086135,0.067618,-0.037833,0.030757,0.025349,-0.108394,-0.094468,0.005770,-0.113117,0.085830,0.000143,-0.072300,0.102706,-0.011340,0.043881,-0.047038,0.083147,-0.012830,0.089993,...,0.071186,0.003295,-0.009214,-0.020037,-0.062383,0.039946,-0.079530,-0.160568,-0.039225,0.016961,-0.032694,0.002606,-0.044518,0.014573,-0.048535,0.017993,-0.067808,0.054576,0.017451,-0.090122,-0.01350

In [38]:
item_item_sim_matrix.columns = user_movie_matrix.T.index
item_item_sim_matrix['movieId'] = user_movie_matrix.T.index
item_item_sim_matrix = item_item_sim_matrix.set_index('movieId')
item_item_sim_matrix.head()

movieId,1,2,3,4,5,6,7,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,31,32,34,35,36,39,41,42,43,44,45,46,...,88140,88163,88744,88810,89492,89745,89864,90866,91500,91529,91542,91630,91658,92259,93840,94864,94959,95510,96079,96610,97304,97752,97913,97921,97938,98809,99114,102125,102445,104841,106487,106489,106782,106920,109374,109487,111362,111759,112556,112852
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.000000,-0.007277,-0.031594,-0.112114,-0.084925,0.131804,-0.054632,-0.087031,0.130114,-0.010739,-0.127292,-0.096353,0.011103,-0.116477,0.046306,0.067544,-0.069773,-0.075580,-0.139184,0.099152,-0.029994,-0.097387,-0.107096,0.052382,-0.023883,-0.070794,0.037616,-0.013090,-0.143816,0.134021,0.104742,0.067002,0.062423,-0.047229,-0.007889,0.008920,-0.037446,-0.147649,-0.027022,-0.092771,...,-0.052596,0.032802,0.010607,-0.008083,0.086686,0.068732,0.082161,0.092138,-0.033797,0.079142,0.009608,0.049843,0.053826,0.017696,0.067804,-0.081896,0.040345,-0.044197,0.084057,0.040065,0.058572,0.020407,0.098499,0.047683,0.040197,-0.026742,0.128330,0.008891,-0.032677,0.038396,0.103224,0.004923,0.061771,-2.853563e-02,0.068199,-0.016748,0.043112,0.115313,0.081266,0.089119
2,-0.007277,0.000000,0.141003,0.017774,0.157326,-0.115083,0.095137,0.086244,0.088405,-0.057072,0.126449,-0.017812,0.038624,0.094194,-0.086791,-0.131311,-0.001828,0.211701,0.063678,0.047955,0.097420,0.035511,0.111818,-0.063916,-0.033053,-0.006995,-0.089168,-0.045563,0.092575,-0.192623,0.046540,0.031395,-0.039137,0.040389,0.018714,0.023227,-0.005611,0.114926,0.042039,-0.006644,...,-0.049050,-0.016471,0.007191,-0.006670,-0.118413,-0.133636,-0.096270,0.010410,-0.037918,-0.043505,-0.093290,-0.015636,-0.033560,-0.018887,-0.060303,0.039396,-0.099282,0.057432,-0.090076,-0.078073,-0.074336,-0.010221,-0.057673,0.017165,0.002624,-0.037733,-0.113207,0.007571,0.004953,-0.153065,-0.079466,-0.021880,-0.107930,-5.311594e-02,-0.082836,-0.135727,-0.064378,-0.073353,-0.077031,-0.107737
3,-0.031594,0.141003,0.000000,0.090117,0.168187,-0.058197,0.098877,0.096959,0.100318,-0.126105,0.119754,-0.122850,0.032978,0.037591,-0.019851,-0.050918,0.026399,0.199380,0.001943,0.070301,0.029817,0.076066,0.038338,-0.054217,0.002246,-0.046317,-0.089217,-0.029053,-0.005643,-0.056524,-0.001915,-0.011630,-0.024059,0.037106,0.007963,0.029700,-0.022484,0.168235,-0.034380,-0.066714,...,-0.012139,-0.005233,0.001949,-0.050156,-0.090194,-0.043842,-0.037210,-0.084131,-0.038575,-0.050541,-0.072345,-0.023248,-0.046942,-0.043564,-0.043742,-0.036170,-0.030675,-0.003238,-0.033010,-0.107466,-0.037243,0.002773,-0.019546,-0.013317,-0.013576,-0.054247,-0.092227,-0.033111,-0.060398,-0.040722,-0.006139,-0.021662,-0.034608,-5.433816e-03,-0.073513,-0.077483,-0.042966,-0.032062,-0.020015,-0.058697
4,-0.112114,0.017774,0.090117,0.000000,0.077437,-0.021579,0.175584,0.073419,0.009188,0.057196,-0.000597,0.010305,0.000238,0.161127,-0.048875,-0.082975,-0.025624,0.152856,0.028948,0.038174,-0.043365,0.012787,-0.048001,-0.186289,-0.017436,0.023842,-0.036042,-0.126064,-0.007546,-0.071052,-0.045153,-0.149667,-0.058492,-0.005848,-0.039359,-0.021668,0.081075,0.054547,0.026525,0.127764,...,0.008995,-0.023751,-0.018682,-0.009570,-0.017160,-0.019711,-0.009586,-0.026425,-0.026799,0.008910,-0.024172,0.000584,-0.029833,-0.008420,-0.013855,-0.036805,-0.124496,0.010260,-0.028603,-0.015729,-0.054039,0.018653,0.012379,-0.049026,-0.011943,-0.015531,-0.121384,0.009509,0.009081,-0.001081,0.006465,0.000134,0.000002,-2.503878e-16,-0.126685,-0.040177,-0.023114,0.006900,0.000470,-0.021005
5,-0.084925,0.157326,0.168187,0.077437,0.000000,-0.120131,0.091224,0.124185,0.034659,0.275990,0.129333,-0.036621,0.033024,0.057356,-0.151201,-0.115386,-0.016743,0.172960,0.052055,0.038355,0.156915,0.086135,0.067618,-0.037833,0.030757,0.025349,-0.108394,-0.094468,0.005770,-0.113117,0.085830,0.000143,-0.072300,0.102706,-0.011340,0.043881,-0.047038,0.083147,-0.012830,0.089993,...,0.071186,0.003295,-0.009214,-0.020037,-0.062383,

Making Recommendations:

In [52]:
movied_id=109487
print('movie name: ', titles.loc[movied_id])
#top_10_similar_items to this item 
n=10
top_10_similar_items = list(
    item_item_sim_matrix\
        .loc[movied_id]\
        .sort_values(ascending=False)\
        .iloc[:n]\
    .index
)

titles.loc[top_10_similar_items]

movie name:  Interstellar (2014)


movieId
106782      Wolf of Wall Street, The (2013)
96610                         Looper (2012)
112852       Guardians of the Galaxy (2014)
104841                       Gravity (2013)
3788                Blow-Up (Blowup) (1966)
68237                           Moon (2009)
2644                         Dracula (1931)
69122                  Hangover, The (2009)
2289                     Player, The (1992)
111362    X-Men: Days of Future Past (2014)
Name: title, dtype: object

In [53]:
movied_id=1
print('movie name: ', titles.loc[movied_id])
#top_10_similar_items to this item 
n=10
top_10_similar_items = list(
    item_item_sim_matrix\
        .loc[movied_id]\
        .sort_values(ascending=False)\
        .iloc[:n]\
    .index
)

titles.loc[top_10_similar_items]

movie name:  Toy Story (1995)


movieId
3114            Toy Story 2 (1999)
4886         Monsters, Inc. (2001)
78499           Toy Story 3 (2010)
6377           Finding Nemo (2003)
4306                  Shrek (2001)
8961       Incredibles, The (2004)
364          Lion King, The (1994)
919       Wizard of Oz, The (1939)
1270     Back to the Future (1985)
150               Apollo 13 (1995)
Name: title, dtype: object